In [1]:
# %matplotlib widget
# %matplotlib inline
%matplotlib qt5

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

from qick import *
from qick.helpers import gauss

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
import scipy as sp
import json
from scipy.fft import fft, fftfreq

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

# Figure params
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 14})
mpl.rcParams['figure.dpi'] = 150

import Pyro4.util

default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle=['solid', 'dashed', 'dotted', 'dashdot']
marker_cycle = ['o', '*', 's', '^']

Could not import QickInstrument (will only work if running on rfsoc). If running on rfsoc, try install/update qick package
Warning failed to import AWG81180A
Warning failed to import Tek5014
Warning failed to import Tek70001
Warning failed to import M8195A


c:\ProgramData\Anaconda3\lib\site-packages\IPython\qt.py:12: ShimWarning: The `IPython.qt` package has been deprecated since IPython 4.0. You should import from qtconsole instead.
  warn("The `IPython.qt` package has been deprecated since IPython 4.0. "


Could not load InstrumentManagerWindow
Could not load labbrick
Could not load Autonics TM4 Driver
Could not load Oxford Trition driver


# Load experiments

In [3]:
# %aimport experiments
# NOTE: ADDING NEW METHODS WILL NOT BE UPDATED
import experiments as meas

imported experiments.clifford_averager_program
imported experiments.fitting
imported experiments.four_qubit.fourQ_state_tomo
imported experiments.qram_protocol_timestepped
imported experiments.single_qubit.acstarkshift_spectroscopy
imported experiments.single_qubit.amplitude_rabi
imported experiments.single_qubit.length_rabi
imported experiments.single_qubit.pulse_probe_spectroscopy
imported experiments.single_qubit.resonator_spectroscopy
imported experiments.single_qubit.rfsoc_tof_calibration
imported experiments.single_qubit.single_shot
imported experiments.single_qubit.t1
imported experiments.single_qubit.t2_ramsey
imported experiments.three_qubit.threeQ_state_tomo
imported experiments.two_qubit.amplitude_rabi_EgGf
imported experiments.two_qubit.amplitude_rabi_f0g1
imported experiments.two_qubit.amprabi_opt_ctrl_state_prep
imported experiments.two_qubit.crosstalk_echo_calib
imported experiments.two_qubit.length_rabi_EgGf
imported experiments.two_qubit.length_rabi_F0G1
imported exper

### Set save data

In [4]:
save_data = False

In [5]:
def prev_data(expt_path, filename=None):
    """Reopen saved data"""
    temp_data_file = expt_path
    if filename is not None: temp_data_file = os.path.join(expt_path, filename)
    print(temp_data_file)
    with SlabFile(temp_data_file) as a:
        attrs = dict()
        for key in list(a.attrs):
            attrs.update({key:json.loads(a.attrs[key])})
        keys = list(a)
        temp_data = dict()
        for key in keys:
            temp_data.update({key:np.array(a[key])})
        print(f'Extracted data and attrs {list(a.attrs)}')
    return temp_data, attrs

In [6]:
# expt_path=os.path.join(os.getcwd(), 'data', 'data_240617')
data_path = 'S:\\QRAM\\qram_4QR2\\'
expt_path = os.path.join(data_path, 'data', 'data_241025')
print('Data will be stored in', expt_path)

Data will be stored in S:\QRAM\qram_4QR2\data\data_241025


In [7]:
# config_file = 'config_zcu216.yml'
# config_file = 'config_q3diamond.yml'
# config_file = 'config_q3diamond_full688_reset.yml'
config_file = 'config_q3diamond_full688and638_reset.yml'
config_path = os.path.join('s:\\Connie\\experiments\\qram_tprocv1_expts\\configs\\', config_file)
print('Config will be', config_path)

Config will be s:\Connie\experiments\qram_tprocv1_expts\configs\config_q3diamond_full688and638_reset.yml


# Load config

In [192]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

# Setup experiment hardware

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [9]:
# im = InstrumentManager(ns_address='192.168.14.1') # Chicago lab
# im = InstrumentManager(ns_address='192.168.137.1') # SLAC lab
# im = InstrumentManager(ns_address='10.108.30.56') # Spilker Fermium
im = InstrumentManager(ns_address='10.108.30.75') # Spilker Vibranium

print(list(im))
# print(im)

['Pyro.NameServer', 'Qick_QRAM']


In [10]:
soc = QickConfig(im[yaml_cfg['aliases']['soc']].get_cfg())
print(soc)
# print(soc.description())
# print(soc.dump_cfg())

# soc = QickConfig(im['Qick187'].get_cfg())

QICK running on ZCU216, software version 0.2.272

Firmware configuration (built Sat Oct  5 10:21:47 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 3 is 3_230, on JHC4
	4:	axis_signal_gen_v6 - envelope memory 32768 samples (4.762 us)
		fs=6881.280 MHz, fabric=430.080 MH

In [11]:
# lo1 = im[yaml_cfg.aliases.readout_LO]
# lo1.open_device()
# print(lo1.get_id())

# # TURN OFF
# lo1.set_standby(True)
# lo1.set_output_state(False)

# # # TURN ON
# # lo1.set_standby(False)
# # lo1.set_output_state(True)
# # lo_freq = float(yaml_cfg.hw.lo.readout.frequency)
# # lo_power = float(yaml_cfg.hw.lo.readout.power)
# # lo1.set_frequency(lo_freq) # Set LO frequency
# # print(f'set freq to [MHz] {lo_freq*1e-6}')
# # lo1.set_power(lo_power) # Set LO power
# # print('set power to', lo_power)

## Set TWPA parameters

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [ ]:
"""
TWPA ON
"""

print('Using config file', config_file)
with open(config_file, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

im = InstrumentManager(ns_address='10.108.30.56') # Spilker Fermium

lo_twpa = im[yaml_cfg.aliases.twpa_LO]
# lo_twpa = SignalCore(name="SignalCore", address='10003494')
lo_twpa.open_device()
print(lo_twpa.get_id())

# TURN ON
lo_twpa.set_standby(False)
lo_twpa.set_output_state(True)
twpa_freq = float(yaml_cfg.hw.lo.twpa.frequency)
twpa_power = float(yaml_cfg.hw.lo.twpa.power)
lo_twpa.set_frequency(twpa_freq) # Set LO frequency
print(f'set twpa pump freq to {twpa_freq*1e-6} MHz')
lo_twpa.set_power(twpa_power) # Set LO power
print(f'set twpa pump power to {twpa_power} dBm')

In [ ]:
"""
TWPA OFF
"""
im = InstrumentManager(ns_address='10.108.30.56') # Spilker Fermium

lo_twpa = im[yaml_cfg.aliases.twpa_LO]
# lo_twpa = SignalCore(name="SignalCore", address='10003494')
lo_twpa.open_device()
print(lo_twpa.get_id())

# TURN OFF
lo_twpa.set_standby(True)
lo_twpa.set_output_state(False)

# Qubit/resonator to probe

In [144]:
all_results = [dict(), dict(), dict(), dict()]
saved_files = []

In [172]:
qubit_i = 3
results_Qi = all_results[qubit_i]

In [146]:
full_mux_expt = False
resonator_reset = [0, 2, 3] if full_mux_expt else None
use_robust_pulses = True

## Set reps and rounds

In [45]:
reps = 100
rounds = 10

# RFSoC Calibration

In [ ]:
tof = meas.ToFCalibrationExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"adc_trig_offset_calibration_qubit{qubit_i}",
    config_file=config_path,
)

tof.cfg.expt = dict(
    pulse_length=0.5, # [us]
    readout_length=1.0, # [us]
    trig_offset=0, # [clock ticks]
    # gain=32000,
    gain=0.8, # [MUX units: -1 to 1]
    frequency=tof.cfg.device.readout.frequency[qubit_i]-15, # [MHz]
    reps=reps*rounds, # Number of averages per point
    # qubit=qubit_i,
    qubit=qubit_i,
)
tof.cfg.device.readout.relax_delay=0.1 # wait time between experiments [us]
# print(tof.cfg)

try:
    tof.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
tof.display(adc_trig_offset=150)

# Initial single tone resonator spectroscopy

In [69]:
rspec = meas.ResonatorSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"resonator_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
rspec.cfg = AttrDict(deepcopy(yaml_cfg))

span = 1.0 # MHz
npts = 150
rspec.cfg.expt = dict(
    start=rspec.cfg.device.readout.frequency[qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps*rounds,
    pulse_e=False, # add ge pi pulse prior to measurement
    pulse_f=False, # add ef pi pulse prior to measurement
    qTest=qubit_i,
    qZZ=None,

    resonator_reset=resonator_reset,
    full_mux_expt=full_mux_expt,
)
# rspec.cfg.device.readout.gain[qubit_i] /= 3
# rspec.cfg.device.readout.readout_length = [3*rspec.cfg.device.readout.readout_length[qubit_i]]*4

rspec.cfg.device.readout.relax_delay = 25 # Wait time between experiments [us] - should be 5x resonator lifetime for good shape

try:
    rspec.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/150 [00:00<?, ?it/s]

In [70]:
# TESTING
# # rspec = meas.ResonatorSpectroscopyExperiment(config_file=config_path,) 
# temp_data, attrs = prev_data(expt_path, '00001_resonator_spectroscopy_qubit2.h5')
# rspec.data = temp_data

rspec.data = rspec.analyze(fit=True)
rspec_fit = rspec.data['fit']
# if not isinstance(rspec_fit, (list, np.ndarray)):
#     rspec.display(fit=False)
#     assert False, 'Fit failed, please manually fix!'
rspec.display(fit=True)
# yaml_cfg.device.readout.frequency[qubit_i] = rspec_fit[0]

res_freq_g = rspec.data['xpts'][np.argmin(rspec.data['amps'])]
# res_freq_g = rspec.data['xpts'][np.argmax(rspec.data['amps'])]
yaml_cfg.device.readout.frequency[qubit_i] = res_freq_g

print(f'*Set cfg res {qubit_i} freq to {yaml_cfg.device.readout.frequency[qubit_i]}*')
results_Qi.update({'Res freq':float(yaml_cfg.hw.lo.readout.frequency)*1e-6 + yaml_cfg.device.readout.lo_sideband[qubit_i]*(yaml_cfg.device.readout.frequency[qubit_i] + yaml_cfg.hw.soc.dacs.readout.mixer_freq[qubit_i])})
results_Qi.update({'Res freq (yaml)':yaml_cfg.device.readout.frequency[qubit_i]})
results_Qi.update({'Res kappa':abs(rspec_fit[0]*(1/rspec_fit[1] + 1/rspec_fit[2]))})
print(f'qubit {qubit_i}', results_Qi)

electrical delay phase correction (no mux support) (deg) -9.296828644341398
param guess [6971.096666666665, 34855.48333333332, 34855.48333333332, 0, -0.05735332374579296, 0.8372734354994826]
param bounds ([6970.606666666667, 0, 0, -6.283185307179586, -0.08602998561868944, -inf], [6971.586666666667, 1000000000.0, 100000000.0, 6.283185307179586, 0.08602998561868944, 1.0161949146025724])
popt [ 6.97112193e+03  8.16747946e+04  6.00218975e+04  3.25146604e-01
  2.87600896e-02 -3.68129692e-02]

Freq with minimum transmission: -228.85333333333335
Freq with maximum transmission: -229.21333333333334
From fit:
	f0: 6971.121925059257
	Qi: 81674.7945879643 	 kappa_i/2pi: 0.08535218190908717
	Qe: 60021.89751385618 	 kappa_e/2pi: 0.11614297804313765
	Q0: 34596.969608163956
	kappa [MHz]: 0.20149515995222483
	phi [radians]: 0.3251466035085062
baseline 119.29965590573188
*Set cfg res 3 freq to -228.84666666666666*
qubit 3 {'Pi pulse gain': 5483, 'Qubit freq': 4381.08536678259, 'Res freq': 6971.153333333

In [21]:
if save_data: saved_files.append(rspec.save_data())

# Qubit ge spectroscopy

## Pulse Probe (qubit) spectroscopy

In [147]:
qspec = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qspec.cfg = AttrDict(deepcopy(yaml_cfg))

span = 50
npts = 250

qspec.cfg.expt = dict(
    start=np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps, # Number of averages per point
    rounds=rounds, # Number of start to finish sweeps to average over
    length=1, # Qubit probe constant pulse length [us]
    gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i]/10), # Qubit pulse gain
    pulse_type='gauss',
    qTest=qubit_i,
    qZZ=None,
    checkEF=False,
)
qspec.cfg.device.readout.relax_delay = 10 # Wait time between experiments [us]
qspec.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [72]:
# # TESTING
# # qspec = meas.PulseProbeSpectroscopyExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00001_qubit_spectroscopy_qubit0.h5')
# qspec.data = temp_data

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    try: 
        qspec.analyze(fit=True, signs=[1, sign, sign])
        fit_err_i = qspec.data['fit_err_avgi'][2][2]
        fit_err_q = qspec.data['fit_err_avgq'][2][2]
        if fit_err_i < best_fit_err_i:
            best_signs[1] = sign
            best_fit_err_i = fit_err_i
        if fit_err_q < best_fit_err_q:
            best_signs[2] = sign
            best_fit_err_q = fit_err_q
    except: continue
if best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qspec.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qspec.analyze(fit=True, signs=best_signs)
qspec.display(fit=True, signs=best_signs)

qspec_fit, qspec_fit_err = meas.fitting.get_best_fit(qspec.data)
yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = qspec_fit[2]
print(f'*Set qubit {qubit_i} f_ge to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 4381.13607506006, HWHM 0.1774060741015645
Found peak in I at [MHz] 4381.159491623125, HWHM -0.09843212084323189
Found peak in Q at [MHz] 4381.135527846996, HWHM -0.17892751746598456
best 2 0.024896841246068727
*Set qubit 3 f_ge to 4381.135527846996*
{'Pi pulse gain': 5483, 'Qubit freq': 4381.135527846996, 'Res freq': 6971.153333333334, 'Res freq (yaml)': -228.84666666666666, 'Res kappa': 0.20149515995222483}


In [24]:
if save_data: saved_files.append(qspec.save_data())

## Pulse Probe (qubit) spectroscopy round 2

In [25]:
qspec2 = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qspec2.cfg = AttrDict(deepcopy(yaml_cfg))

span = 15
npts = 250

qspec2.cfg.expt = dict(
    start=np.reshape(qspec2.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps, # Number of averages per point
    rounds=rounds, # Number of start to finish sweeps to average over
    length=1, # Qubit probe constant pulse length [us]
    gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i]/80), # Qubit pulse gain
    pulse_type='gauss',
    qTest=qubit_i,
    qZZ=None,
    checkEF=False,
)
qspec2.cfg.device.readout.relax_delay = 10 # Wait time between experiments [us]
qspec2.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [26]:
# # TESTING
# # qspec2 = meas.PulseProbeSpectroscopyExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00001_qubit_spectroscopy_qubit0.h5')
# qspec2.data = temp_data

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    try: 
        qspec2.analyze(fit=True, signs=[1, sign, sign])
        fit_err_amps = qspec2.data['fit_err_amps'][2][2]
        fit_err_i = qspec2.data['fit_err_avgi'][2][2]
        fit_err_q = qspec2.data['fit_err_avgq'][2][2]
        if fit_err_i < best_fit_err_i:
            best_signs[1] = sign
            best_fit_err_i = fit_err_i
        if fit_err_q < best_fit_err_q:
            best_signs[2] = sign
            best_fit_err_q = fit_err_q
    except: continue
if fit_err_amps == np.inf and best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qspec2.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qspec2.analyze(fit=True, signs=best_signs)
qspec2.display(fit=True, signs=best_signs)

qspec2_fit, qspec2_fit_err = meas.fitting.get_best_fit(qspec2.data)
yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = qspec2_fit[2]
qubit_hwhm = qspec2_fit[3]
print(f'*Set qubit {qubit_i} f_ge to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 4381.000276870876, HWHM 2.09882347179696
Found peak in I at [MHz] 4381.084606469684, HWHM 2.2129218497450216
Found peak in Q at [MHz] 4380.640636505068, HWHM 2.369674644626421
best 0 0.038706615167742736
*Set qubit 3 f_ge to 4381.000276870876*
{'Res freq': 6971.086666666667, 'Res freq (yaml)': -278.91333333333296, 'Res kappa': 0.19749745591104786, 'Qubit freq': 4381.000276870876}


In [27]:
if save_data: saved_files.append(qspec2.save_data())

## Length Rabi

In [28]:
lengthrabi = meas.LengthRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"length_rabi_qubit{qubit_i}",
    config_file=config_path,
)
lengthrabi.cfg = AttrDict(deepcopy(yaml_cfg))

lengthrabi.cfg.expt = dict(
    start=0, # pulse length (sigma) [us]
    step=soc.cycles2us(1), # [us]
    # step=0.001,
    expts=121,
    reps=reps*rounds,
    pulse_type='gauss',
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
)
# lengthrabi.cfg.device.readout.relax_delay = 50 # Wait time between experiments [us]
# lengthrabi.pi_ge.gain[4*qubit_i+qubit_i] = 5000 # qubit gain [DAC units],

try:
    lengthrabi.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

Running length rabi on Q3 


  0%|          | 0/121 [00:00<?, ?it/s]

In [29]:
# # TESTING
# # lengthrabi = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# lengthrabi.data = temp_data

lengthrabi.analyze(fit=True)

lengthrabi_fit, lengthrabi_fit_err = meas.fitting.get_best_fit(lengthrabi.data) #, fitfunc=fitter.decaysin)

lengthrabi.display(fit=True)
# p = lengthrabi_fit
# if p[2] > 180: p[2] = p[2] - 360
# elif p[2] < -180: p[2] = p[2] + 360
# if p[2] < 0: pi_len = (1/2 - p[2]/180)/2/p[1]
# else: pi_len = (3/2 - p[2]/180)/2/p[1]
# pi_len = np.max((0.9*pi_len, 0.015)) # since we run the length rabi at a < max gain

# lengthrabi_decay = p[3]

# print(f'Will use sigma {pi_len} for amplitude rabi')

best 1 0.2230375028289761
Pi length from avgi data [us]: 0.05190566156903359
	Pi/2 length from avgi data [us]: 0.025952830784516795

Pi length from avgq data [us]: 0.04876758588147374
Pi/2 length from avgq data [us]: 0.02438379294073687


In [ ]:
# if save_data: saved_files.append(lengthrabi.save_data())

## Amplitude Rabi

In [96]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_len', pi_len)

qubit 3 pi_len 0.025


In [100]:
amprabi = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
npts = 50

amprabi.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps*rounds,
    rounds=1,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    # sigma_test=0.2,
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
    # pulse_type='gauss', 
    pulse_type='robust',

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=use_robust_pulses,
)
amprabi.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi on Q3 


  0%|          | 0/50000 [00:00<?, ?it/s]

(4, 50, 1000)


In [101]:
# # TESTING
# # amprabi = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi.data = temp_data

# amprabi.display(fit=False)

amprabi.analyze(fit=True)

amprabi_fit, amprabi_fit_err = meas.fitting.get_best_fit(amprabi.data)

p = amprabi_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi.display(fit=False)
    print(f'WARNING: Fit pi pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi.display(fit=True)
yaml_cfg.device.qubit.pulses.pi_ge.sigma[qubit_i] = amprabi.cfg.expt.sigma_test

print(f'*Set pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ge.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]})
print(f'*New pi_ge.gain\n', yaml_cfg.device.qubit.pulses.pi_ge.gain)
yaml_cfg.device.qubit.pulses.pi_ge.gain = yaml_cfg.device.qubit.pulses.pi_ge.gain.flatten().astype(int).tolist()
print(f'*New pi_ge.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(results_Qi)

best 1 0.03829799089225851
Pi gain from amps data [dac units]: 14364
	Pi/2 gain from amps data [dac units]: 7182
Pi gain from avgi data [dac units]: 14360
	Pi/2 gain from avgi data [dac units]: 7180
Pi gain from avgq data [dac units]: 14412
	Pi/2 gain from avgq data [dac units]: 7206
*Set pi pulse sigma for qubit 3 to 0.025*
*Set pi pulse gain for qubit 3 to 14360*
*New pi_ge.gain
 [[ 1849  1914     0  1855]
 [ 2834  2504  2648  2631]
 [ 9155  9155  9155  9155]
 [ 2000  2000  2000 14360]]
*New pi_ge.gain flattened
[1849, 1914, 0, 1855, 2834, 2504, 2648, 2631, 9155, 9155, 9155, 9155, 2000, 2000, 2000, 14360]
{'Pi pulse gain': 14360, 'Qubit freq': 4381.135527846996, 'Res freq': 6971.153333333334, 'Res freq (yaml)': -228.84666666666666, 'Res kappa': 0.20149515995222483}


In [26]:
if save_data: saved_files.append(amprabi.save_data())

## $T_1$

In [193]:
short_T1 = False
# short_T1 = True

In [195]:
t1 = meas.T1Experiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"t1_qubit{qubit_i}",
    config_file=config_path,
)
t1.cfg = AttrDict(deepcopy(yaml_cfg))

span = 50 if short_T1 else 400
npts = 151

t1.cfg.expt = dict(
    start=0, # wait time [us]
    step=span/npts,
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    qTest=qubit_i,
    qZZ=None,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=use_robust_pulses,
)
t1.cfg.device.readout.relax_delay = span # Wait time between experiments [us]
t1.go(analyze=False, display=False, progress=True, save=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [196]:
# # TESTING
# # t1 = meas.T1Experiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_t1_qubit1.h5')
# t1.data = temp_data

t1.analyze(fit_log=False)

t1_fit, t1_fit_err = meas.fitting.get_best_fit(t1.data) #, fitter.expfunc)

t1.display(fit=True, fit_log=False)
t1_qubit = t1_fit[3]
yaml_cfg.device.readout.relax_delay[qubit_i] = np.round(4*t1_qubit)
print(f'*Set readout {qubit_i} relax delay to {yaml_cfg.device.readout.relax_delay[qubit_i]}*')
results_Qi.update({'T1':t1_qubit, 'T1 err':np.sqrt(t1_fit_err[3][3])})
print(f'qubit {qubit_i}', results_Qi)

best 1 78107.60401250764
Fit T1 amps [us]: 49.4285115667099
Fit T1 avgi [us]: 55.69833035027093
Fit T1 avgq [us]: 54.73630001681735
*Set readout 3 relax delay to 223.0*
qubit 3 {'Qubit EF freq': 4203.908838259012, 'T2R EF': 0.2078683035714286, 'T2R EF err': 0.0639209751136206, 'Pi/2 EF pulse gain': 3650, 'T1': 55.69833035027093, 'T1 err': 2.013371053265863}


 s:\Connie\experiments\qram_tprocv1_expts\experiments\fitting.py: 94RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
 s:\Connie\experiments\qram_tprocv1_expts\experiments\single_qubit\t1.py: 184

In [30]:
if save_data: saved_files.append(t1.save_data())

## $T_2$ Ramsey round 1

In [139]:
# yaml_cfg.device.qubit.f_ge[qubit_i] = 145.572034953271
qubit_hwhm = 2
short_T1 = False
# short_T1 = t1_qubit < 20
print('Short T1', short_T1)

Short T1 False


In [ ]:
# ramsey_freq = max((min((np.abs(np.ceil(qubit_hwhm)), 3)), 2))
ramsey_freq = 3.0
print(f'Ramsey freq [MHz]: {ramsey_freq}')

Ramsey freq [MHz]: 3.0


In [19]:
t2r = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"ramsey_qubit{qubit_i}",
    config_file=config_path,
)
t2r.cfg = AttrDict(deepcopy(yaml_cfg))

# make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
t2r.cfg.expt = dict(
    start=0, # wait time tau [us]
    step=soc.cycles2us(8), # [us] 
    # step=soc.cycles2us(2) if short_T1 else soc.cycles2us(10), # [us] 
    expts=200,
    ramsey_freq=ramsey_freq, # [MHz]
    reps=reps,
    rounds=rounds,
    qTest=qubit_i,
    checkEF=False,
    qZZ=None,
)

try:
    t2r.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

mux_gains [0.5, 0, 0.4, 0.3]
Running Ramsey on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [20]:
# # TESTING
# # t2r = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_ramsey_qubit2.h5')
# t2r.data = temp_data

# t2r.analyze(fit=True, fit_num_sin=2)
# t2r.display(fit=True, fit_num_sin=2)

t2r.analyze(fit=True)
t2r_fit, t2r_fit_err, t2r_adjust = meas.fitting.get_best_fit(t2r.data, get_best_data_params=['f_adjust_ramsey'])

old_qubit_freq = np.reshape(t2r.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]
t2r.display(fit=True)
yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = old_qubit_freq + t2r_adjust[0]
alt_qubit_freq = old_qubit_freq + t2r_adjust[1]
results_Qi.update({'T2R':t2r_fit[3], 'T2R err':np.sqrt(t2r_fit_err[3][3])})
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
print(f'Alt qubit freq is {alt_qubit_freq}')

print(f'*New f_ge\n', yaml_cfg.device.qubit.f_ge)
yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
print(f'*New f_ge flattened\n{yaml_cfg.device.qubit.f_ge}')

data = t2r.data
fit_axis = 'avgq'
xpts = data['xpts']
fourier = np.fft.fftshift(np.abs(np.fft.fft(data[fit_axis])))
freqs = np.fft.fftshift(np.fft.fftfreq(len(fourier), d=(xpts[1]-xpts[0])))
plt.plot(freqs, fourier)
ZZs = np.zeros((4,4))
reshaped_f_ge = np.reshape(t2r.cfg.device.qubit.f_ge, (4,4))
for row in range(4):
    ZZs[row,:] = reshaped_f_ge[row,:] - reshaped_f_ge[row,row]
for i in range(4):
    if (qubit_i+i)%4 == qubit_i: color = 'r'
    else: color = 'k'
    plt.axvline(t2r.cfg.expt.ramsey_freq - ZZs[qubit_i, (qubit_i+i)%4], color=color, linewidth=0.5, linestyle='--')
plt.xlabel('Frequency [MHz]')
plt.xlim(-6, 6)
# plt.ylim(0, 20000)
plt.title(f'Fourier Transform of Ramsey on Q{qubit_i} (Ramsey Freq: {t2r.cfg.expt.ramsey_freq})')
plt.show()

best 0 0.07678762677808101
Current pi pulse frequency: 4381.200598328156
Fit frequency from amps [MHz]: 2.9195455320802473 +/- 0.007359468434210232
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4381.2810527960755
 	4375.2810527960755
T2 Ramsey from fit amps [us]: 5.217930655758489 +/- 1.2362424540001307
Current pi pulse frequency: 4381.200598328156
Fit frequency from I [MHz]: 2.9062430346083525 +/- 0.001389192115092896
Suggested new pi pulse frequency from fit I [MHz]:
 	4381.294355293548
 	4375.294355293548
T2 Ramsey from fit I [us]: 53.95653762310775 +/- 24.953736398771674
Fit frequency from Q [MHz]: 2.9071384499619395 +/- 0.001683897690362356
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4381.293459878194
 	4375.293459878194
T2 Ramsey from fit Q [us]: 40.616560721372196 +/- 17.145175664355193
*Set qubit 3 freq to 4381.2810527960755*
Alt qubit freq is 4375.2810527960755
*New f_ge
 [[4111.14452003 4105.78395881 4111.00289644 4110.59669631]
 [3442.97955523 3448.348

In [40]:
if save_data: saved_files.append(t2r.save_data())

## Amplitude Rabi round 2

In [24]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
print('qubit', qubit_i, 'pi_len', pi_len)
# pi_len = 0.025

qubit 0 pi_len 0.02


In [25]:
amprabi_2 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_2.cfg = AttrDict(deepcopy(yaml_cfg))

n_pulses = 1
span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_2.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss',
    checkEF=False, 
    n_pulses=n_pulses,
    qTest=qubit_i,
    qZZ=None,
)
amprabi_2.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi on Q0 


  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# # TESTING
# # amprabi_2 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_2.data = temp_data

amprabi_2.analyze(fit=True)

amprabi_2_fit, amprabi_2_fit_err = meas.fitting.get_best_fit(amprabi_2.data)

p = amprabi_2_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_2.display(fit=False)
    print(f'WARNING: Fit pi pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_2.display(fit=True)
yaml_cfg.device.qubit.pulses.pi_ge.sigma[qubit_i] = amprabi_2.cfg.expt.sigma_test

print(f'*Set pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ge.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]})
print(f'*New pi_ge.gain\n', yaml_cfg.device.qubit.pulses.pi_ge.gain)
yaml_cfg.device.qubit.pulses.pi_ge.gain = yaml_cfg.device.qubit.pulses.pi_ge.gain.flatten().tolist()
print(f'*New pi_ge.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(results_Qi)

best 0 0.022292039093864034
Pi gain from amps data [dac units]: 935
	Pi/2 gain from amps data [dac units]: 467
Pi gain from avgi data [dac units]: 935
	Pi/2 gain from avgi data [dac units]: 467
Pi gain from avgq data [dac units]: 936
	Pi/2 gain from avgq data [dac units]: 468
*Set pi pulse sigma for qubit 3 to 0.025*
*Set pi pulse gain for qubit 3 to 935*
*New pi_ge.gain
 [[ 1849  1914     0  1855]
 [ 2834  2504  2648  2631]
 [14489 14572 14736 14639]
 [  963   959   974   935]]
*New pi_ge.gain flattened
[1849, 1914, 0, 1855, 2834, 2504, 2648, 2631, 14489, 14572, 14736, 14639, 963, 959, 974, 935]
{'T1': 52.3580045090926, 'T1 err': 1.6646251872930162, 'T2R': 28.080322737694868, 'T2R err': 10.12437756373936, 'Qubit freq': 4381.110012654556, 'Pi pulse gain': 935}


In [38]:
if save_data: saved_files.append(amprabi_2.save_data())

## $T_2$ Ramsey round 2

Check that frequency adjustment helps.

In [141]:
t2r_2 = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"ramsey_qubit{qubit_i}",
    config_file=config_path,
)
t2r_2.cfg = AttrDict(deepcopy(yaml_cfg))

# make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
t2r_2.cfg.expt = dict(
    start=0, # wait time tau [us]
    # step=soc.cycles2us(2) if t2r_fit[3] < 3 else soc.cycles2us(60), # [us]
    step=soc.cycles2us(150), # [us]
    expts=151,
    ramsey_freq=ramsey_freq/15, # [MHz]
    reps=reps*rounds*2,
    rounds=1,
    checkEF=False,
    qTest=qubit_i,
    qZZ=None,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)
t2r_2.go(analyze=False, display=False, progress=True, save=False)

Running Ramsey on Q3 


  0%|          | 0/302000 [00:00<?, ?it/s]

In [142]:
# TESTING
# t2r_2 = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00004_ramsey_qubit2.h5')
# t2r_2.data = temp_data
# t2r_2.cfg = AttrDict(attrs['config'])

redo_t2r = True

t2r_2.analyze(fit=True)

t2r_2_fit, t2r_2_fit_err, t2r_2_adjust = meas.fitting.get_best_fit(t2r_2.data, get_best_data_params=['f_adjust_ramsey'])

t2r_2.display(fit=True)
old_qubit_freq = np.reshape(t2r_2.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i]
if abs(t2r_2_adjust[0]) <= ramsey_freq/10:
# if abs(t2r_2_adjust[0]) <= abs(t2r_2_adjust[0]):
    redo_t2r = False
    yaml_cfg.device.qubit.f_ge = np.reshape(yaml_cfg.device.qubit.f_ge, (4,4))
    yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] = old_qubit_freq + t2r_2_adjust[0]
    results_Qi.update({'T2R':t2r_2_fit[3], 'T2R err':np.sqrt(t2r_2_fit_err[3][3])})
    results_Qi.update({'Qubit freq':yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
    print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit.f_ge[qubit_i, qubit_i]}*')
    print(results_Qi)
    print(f'*New f_ge\n', yaml_cfg.device.qubit.f_ge)
    yaml_cfg.device.qubit.f_ge = yaml_cfg.device.qubit.f_ge.flatten().tolist()
    print(f'*New f_ge flattened\n{yaml_cfg.device.qubit.f_ge}')

else: # redo_t2r = True
    yaml_cfg.device.qubit.f_ge[qubit_i] = alt_qubit_freq 
    print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit.f_ge[qubit_i]}*')
print(f'Redo T2R: {redo_t2r}')

best 1 0.07262885928179506
Current pi pulse frequency: 4381.537145425829
Fit frequency from amps [MHz]: 0.21144705229716315 +/- 0.0007987366753312534
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4381.525698373533
 	4381.125698373532
T2 Ramsey from fit amps [us]: 43.30295469865911 +/- 9.176970091963625
Current pi pulse frequency: 4381.537145425829
Fit frequency from I [MHz]: 0.21139380568504612 +/- 0.0007892314121604746
Suggested new pi pulse frequency from fit I [MHz]:
 	4381.525751620145
 	4381.125751620144
T2 Ramsey from fit I [us]: 41.97569834709405 +/- 8.52088920763707
Fit frequency from Q [MHz]: 0.2113189067633976 +/- 0.0008134761603302702
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4381.525826519066
 	4381.125826519066
T2 Ramsey from fit Q [us]: 36.63047448377203 +/- 6.679117114735598
*Set qubit 3 freq to 4381.525751620145*
{'Pi pulse gain': 14360, 'Qubit freq': 4381.525751620145, 'Res freq': 6971.153333333334, 'Res freq (yaml)': -228.84666666666666, 'Res 

In [43]:
if save_data: saved_files.append(t2r_2.save_data())

## $\pi$, -$\pi$ experiment

In [130]:
use_robust_pulses = True

In [ ]:
piminuspi = meas.PiMinusPiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPiExpt{qubit_i}",
    config_file=config_path,
)

piminuspi.cfg = AttrDict(deepcopy(yaml_cfg))

span_f = 2.0
# npts_f = 50
npts_f = 30

if not use_robust_pulses:
    start_f = piminuspi.cfg.device.qubit.f_ge[qubit_i*4 + qubit_i]-span_f/2
    pulse_type = "gauss"
else:
    start_f=piminuspi.cfg.device.qubit.f_ge_robust[qubit_i*4 + qubit_i]-span_f/2,
    pulse_type = "robust"

piminuspi.cfg.expt = dict(
    start_N=0, # number gate repetitions
    step_N=4,
    expts_N=10,
    start_f=start_f,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,
    reps=int(0.75*reps*rounds),
    # reps=10,
    loops=1,
    pulse_type=pulse_type,
    use_robust_pulses=use_robust_pulses,

    checkEF=False,
    qTest=qubit_i,
    qZZ=None,

    test_pi_half=False,

    post_process='threshold',
    singleshot_reps=15000,
)

# print(piminuspi.cfg)
import Pyro4.util
try:
    piminuspi.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (0)
ge fidelity (%): 78.30666666666667 	 angle (deg): -96.83569591568404 	 threshold ge: 398.7212265605594


  0%|          | 0/10 [00:00<?, ?it/s]

In [132]:
# # TESTING
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00000_PiMinusPiExpt0.h5')
# piminuspi.data =  temp_data
# piminuspi.cfg = AttrDict(attrs['config'])


data = piminuspi.data
piminuspi.analyze(fit=True)
piminuspi.display(fit=True)

if not use_robust_pulses:
    freq_update = "f_ge"
else:
    freq_update = "f_ge_robust"
yaml_cfg.device.qubit[freq_update] = np.reshape(yaml_cfg.device.qubit[freq_update], (4,4))
yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] = data["best_freq"]
results_Qi.update({'Qubit freq':yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i]}*')
print(results_Qi)
print(f'*New {freq_update}\n', yaml_cfg.device.qubit[freq_update])
yaml_cfg.device.qubit[freq_update] = yaml_cfg.device.qubit[freq_update].flatten().tolist()
print(f'*New {freq_update} flattened\n{yaml_cfg.device.qubit[freq_update]}')

Fit best freq 4111.8334058569635 which is -0.47307206386085454 away from old freq 4112.306477920824
*Set qubit 0 freq to 4111.8334058569635*
{'T1': 64.05955000251987, 'T1 err': 1.119240285515182, 'T2R': 18.248886781332068, 'T2R err': 0.7816516069106116, 'Qubit freq': 4111.8334058569635, 'T2E': 56.36959417286505, 'T2E err': 1.4283912853884384, 'Res freq': 6810.011666666666, 'Res freq (yaml)': -389.98833333333334, 'Res kappa': 0.2735895597906132, 'Res chi shift (e - g)': -0.8642384105960446, 'Pi/2 pulse gain': 7569}
*New f_ge_robust
 [[4111.83340586 4112.30647792 4112.30647792 4112.30647792]
 [3450.05408931 3450.05408931 3450.05408931 3450.05408931]
 [4763.941      4763.941      4763.941      4763.941     ]
 [4384.25034226 4384.25034226 4384.25034226 4384.25034226]]
*New f_ge_robust flattened
[4111.8334058569635, 4112.306477920824, 4112.306477920824, 4112.306477920824, 3450.0540893078382, 3450.0540893078382, 3450.0540893078382, 3450.0540893078382, 4763.941, 4763.941, 4763.941, 4763.941, 

In [ ]:
if save_data: piminuspi.save_data()

## GE error amplification

In [ ]:
npulsecalib = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptQ{qubit_i}",
    config_file=config_path,
)

npulsecalib.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=20,
    pulse_type='gauss',
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
    test_pi_half=False,
    error_amp=True,

    post_process='scale',
    singleshot_reps=15000,
)

# print(npulsecalib.cfg)
import Pyro4.util
try:
    npulsecalib.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib.data
npulsecalib.analyze(fit=True, scale=None)
npulsecalib.display(fit=True, scale=None)

# npulsecalib_fit, npulsecalib_fit_err = meas.fitting.get_best_fit(npulsecalib.data)
fit_axis = 'avgi'
npulsecalib_fit = data['fit_avgi']
npulsecalib_fit_err = data['fit_err_avgi']

angle_err = npulsecalib_fit[1]
amp_ratio = (180 - angle_err) / 180
new_amp_ge = np.reshape(npulsecalib.cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ge.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i] = int(round(new_amp_ge))
print(f'*Set pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.gain[qubit_i, qubit_i]})
print(f'*New pi_ge.gain\n', yaml_cfg.device.qubit.pulses.pi_ge.gain)
yaml_cfg.device.qubit.pulses.pi_ge.gain = yaml_cfg.device.qubit.pulses.pi_ge.gain.flatten().tolist()
print(f'*New pi_ge.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(results_Qi)

In [ ]:
if save_data: npulsecalib.save_data()

## $T_2$ Echo

In [26]:
# yaml_cfg.device.qubit.f_ge[qubit_i] = 145.572034953271
qubit_hwhm = 2
short_T1 = False
# short_T1 = True
# short_T1 = yaml_cfg.device.qubit.T1[qubit_i] < 20
print('Short T1', short_T1)

Short T1 False


In [27]:
ramsey_freq = max((min((np.abs(np.ceil(qubit_hwhm)), 3)), 2))
# ramsey_freq = 1
print(f'Ramsey freq [MHz]: {ramsey_freq}')

Ramsey freq [MHz]: 2.0


In [28]:
t2e = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"echo_qubit{qubit_i}",
    config_file=config_path,
)
t2e.cfg = AttrDict(deepcopy(yaml_cfg))

t2e.cfg.expt = dict(
    start=0, # total wait time b/w the two pi/2 pulses [us]
    # step=soc.cycles2us(80), # make sure nyquist freq = 0.5*(1/step) > ramsey (signal) freq!
    step=soc.cycles2us(30) if short_T1 else soc.cycles2us(150), # [us] 
    expts=201,
    ramsey_freq=max((ramsey_freq/15, 0.100)), # frequency by which to advance phase [MHz]
    num_pi=1, # number of pi pulses
    echo_type='cpmg', # cp or cpmg
    reps=reps,
    # reps=10,
    rounds=rounds,
    qTest=qubit_i,
    qZZ=None,
    checkEF=False,
)
t2e.go(analyze=False, display=False, progress=True, save=False)

Running Ramsey Echo on Q0 


  0%|          | 0/10 [00:00<?, ?it/s]

In [29]:
# # TESTING
# # t2e = meas.RamseyEchoExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_echo_qubit1.h5')
# t2e.data = temp_data

print(f'Qubit {qubit_i}')
t2e.analyze(fit=True)

t2e_fit, t2e_fit_err = meas.fitting.get_best_fit(t2e.data)

if not isinstance(t2e_fit, (list, np.ndarray)):
    t2e.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

t2e.display(fit=True)
results_Qi.update({'T2E':t2e_fit[3], 'T2E err':np.sqrt(t2e_fit_err[3][3])})
print(results_Qi)
# print('Results updated.')

Qubit 0
best 0 0.009515671840636859
Current pi pulse frequency: 4111.122260451728
Fit frequency from amps [MHz]: 0.13321301485526182 +/- 7.214999297595147e-05
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4111.122380770206
 	4110.855714103539
T2 Ramsey from fit amps [us]: 56.36959417286505 +/- 1.4283912853884384
Current pi pulse frequency: 4111.122260451728
Fit frequency from I [MHz]: 0.13315630305524895 +/- 7.78367439779741e-05
Suggested new pi pulse frequency from fit I [MHz]:
 	4111.122437482006
 	4110.855770815339
T2 Ramsey from fit I [us]: 56.049642889050325 +/- 1.523004844518435
Fit frequency from Q [MHz]: 0.13330409486482314 +/- 8.258644157305862e-05
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4111.122289690196
 	4110.85562302353
T2 Ramsey from fit Q [us]: 56.803008426451235 +/- 1.6613331354971312
{'T1': 64.05955000251987, 'T1 err': 1.119240285515182, 'T2R': 18.248886781332068, 'T2R err': 0.7816516069106116, 'Qubit freq': 4111.122260451728, 'T2E': 56.36959

In [46]:
if save_data: saved_files.append(t2e.save_data())

## (Divide len) Calibrate $\pi/2$ pulse

### Amplitude rabi for $\pi/2$ pulse (divide len) (use this for protocol)

In [127]:
divide_len = True

In [128]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_len', pi_len)

qubit 3 pi_len 0.025


In [129]:
amprabi_3 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_3.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
npts = 50
    
amprabi_3.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps*rounds,
    rounds=1,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    pulse_type="gauss" if not use_robust_pulses else "robust",
    checkEF=False, 
    n_pulses=1,
    test_pi_half=True,
    qTest=qubit_i,
    qZZ=None,

    use_robust_pulses=use_robust_pulses,
)
amprabi_3.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi on Q3 


  0%|          | 0/50000 [00:00<?, ?it/s]

(4, 50, 1000)


In [130]:
# # TESTING
# # amprabi_3 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_3.data = temp_data

# amprabi_3.display(fit=False)

amprabi_3.analyze(fit=True)

amprabi_3_fit, amprabi_3_fit_err = meas.fitting.get_best_fit(amprabi_3.data)

p = amprabi_3_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_3.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_3.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ge.half_gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.half_gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.half_gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.half_gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.half_gain[qubit_i, qubit_i]})
print(f'*New pi_ge.half_gain\n', yaml_cfg.device.qubit.pulses.pi_ge.half_gain)
yaml_cfg.device.qubit.pulses.pi_ge.half_gain = yaml_cfg.device.qubit.pulses.pi_ge.half_gain.flatten().astype(int).tolist()
print(f'*New pi_ge.half_gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain}')
print(results_Qi)

best 0 0.03736597231018949
Pi gain from amps data [dac units]: 3120
	Pi/2 gain from amps data [dac units]: 1560
Pi gain from avgi data [dac units]: 3120
	Pi/2 gain from avgi data [dac units]: 1560
Pi gain from avgq data [dac units]: 2142
	Pi/2 gain from avgq data [dac units]: 1071
*Set pi/2 pulse gain for qubit 3 to 3120*
*New pi_ge.half_gain
 [[1850 1835 1873 1867]
 [2382 2186 2248 2289]
 [9253 9253 9253 9253]
 [2000 2000 2000 3120]]
*New pi_ge.half_gain flattened
[1850, 1835, 1873, 1867, 2382, 2186, 2248, 2289, 9253, 9253, 9253, 9253, 2000, 2000, 2000, 3120]
{'Pi pulse gain': 14360, 'Qubit freq': 4381.135527846996, 'Res freq': 6971.153333333334, 'Res freq (yaml)': -228.84666666666666, 'Res kappa': 0.20149515995222483, 'Pi/2 pulse gain': 3120}


In [ ]:
if save_data: saved_files.append(amprabi_3.save_data())

### $\pi/2$, -$\pi/2$ experiment

In [113]:
use_robust_pulses = False

In [21]:
sweep_phase = True

In [40]:
piminuspi = meas.PiMinusPiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPiExpt{qubit_i}",
    config_file=config_path,
)

piminuspi.cfg = AttrDict(deepcopy(yaml_cfg))

span_f = 5.0
npts_f = 30
# span_f = 2.0
# npts_f = 10

# span_f = 0.0
# npts_f = 1

start_f = piminuspi.cfg.device.qubit.f_ge[qubit_i*4 + qubit_i]-span_f/2
pulse_type = "gauss"
if use_robust_pulses:
    pulse_type = "robust"
    if not sweep_phase:
        start_f = piminuspi.cfg.device.qubit.f_ge_robust[qubit_i*4 + qubit_i]-span_f/2,
    else:
        start_f = -span_f/2

piminuspi.cfg.expt = dict(
    start_N=0, # number gate repetitions
    step_N=1,
    expts_N=10,
    # step_N=4,
    # expts_N=4,
    start_f=start_f,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,
    # reps=int(0.75*reps*rounds),
    # reps=1000,
    reps=500,
    # reps=10,
    loops=1,
    pulse_type=pulse_type,
    use_robust_pulses=use_robust_pulses,

    checkEF=False,
    qTest=qubit_i,
    qZZ=None,

    # use_Y=True,

    test_pi_half=True,

    post_process='threshold',
    singleshot_reps=15000,

    sweep_phase=sweep_phase,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# print(piminuspi.cfg)
import Pyro4.util
try:
    piminuspi.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (3)
ge fidelity (%): 66.23234437851413 	 angle (deg): -26.387257959233988 	 threshold ge: 52.10000869145318


  0%|          | 0/10 [00:00<?, ?it/s]

In [41]:
# # TESTING
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00000_PiMinusPiExpt0.h5')
# piminuspi.data =  temp_data
# piminuspi.cfg = AttrDict(attrs['config'])


data = piminuspi.data
piminuspi.analyze(fit=True)
piminuspi.display(fit=True)

if not sweep_phase:
    if not use_robust_pulses:
        freq_update = "f_ge"
    else:
        freq_update = "f_ge_robust"
    yaml_cfg.device.qubit[freq_update] = np.reshape(yaml_cfg.device.qubit[freq_update], (4,4))
    yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] = data["best_freq"]
    results_Qi.update({'Qubit freq':yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
    print(f'*Set qubit {qubit_i} freq to {yaml_cfg.device.qubit[freq_update][qubit_i, qubit_i]}*')
    print(results_Qi)
    print(f'*New {freq_update}\n', yaml_cfg.device.qubit[freq_update])
    yaml_cfg.device.qubit[freq_update] = yaml_cfg.device.qubit[freq_update].flatten().tolist()
    print(f'*New {freq_update} flattened\n{yaml_cfg.device.qubit[freq_update]}')

else:
    if not use_robust_pulses:
        yaml_cfg.device.qubit.pulses.pi_ge.half_correction_phase[qubit_i*4 + qubit_i] = data["best_phase"]
        print(f'*New pi_ge.half_correction_phase \n{yaml_cfg.device.qubit.pulses.pi_ge.half_correction_phase}')
    else:
        yaml_cfg.device.qubit.pulses.pihalf_ge_robust.correction_phase[qubit_i*4 + qubit_i] = data["best_phase"]
        print(f'*New pihalf_ge_robust.correction_phase \n{yaml_cfg.device.qubit.pulses.pihalf_ge_robust.correction_phase}')

Fit best freq 1.6904291867323187 which is 1.6904291867323187 away from old freq 0
Best virtual Z (deg) 15.21386268059087
*New pihalf_ge_robust.correction_phase 
[12.831628835545475, 10.05593135477756, 0, 0, 0.11174784781734694, -0.41895542828054083, 0, 0, 0, 0, 2.9582000054694726, 0, 0, 0, 0, 15.21386268059087]


In [ ]:
if save_data: piminuspi.save_data()

### GE $\pi/2$ error amplification (divide len)

In [196]:
divide_len = True

In [197]:
npulsecalib_gehalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptQ{qubit_i}",
    config_file=config_path,
)

if not use_robust_pulses:
    pulse_type = "gauss"
else:
    pulse_type = "robust"

npulsecalib_gehalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_gehalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=20,
    reps=1000,
    loops=20,
    pulse_type=pulse_type,
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,
    use_robust_pulses=use_robust_pulses,
    use_Y=False,

    checkEF=False,
    qTest=qubit_i,
    qZZ=None,

    post_process='scale',
    singleshot_reps=15000,
)

# print(npulsecalib_gehalf.cfg)
try:
    npulsecalib_gehalf.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (3)
ge fidelity (%): 81.28 	 angle (deg): 34.05335509852408 	 threshold ge: -666.1487237776781


  0%|          | 0/20 [00:00<?, ?it/s]

 s:\Connie\experiments\qram_tprocv1_expts\experiments\clifford_averager_program.py: 1742

In [202]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_gehalf.data
npulsecalib_gehalf.analyze(fit=True, scale=None)
npulsecalib_gehalf.display(fit=True, scale=None)

# npulsecalib_gehalf_fit, npulsecalib_gehalf_fit_err = meas.fitting.get_best_fit(npulsecalib_gehalf.data)
fit_axis = 'avgi'
npulsecalib_gehalf_fit = data['fit_avgi']
npulsecalib_gehalf_fit_err = data['fit_err_avgi']

angle_err = npulsecalib_gehalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ge = npulsecalib_gehalf.cfg.expt.gain / amp_ratio

if npulsecalib_gehalf.cfg.expt.use_robust_pulses:
    yaml_update = ["pihalf_ge_robust", "gain"]
else:
    yaml_update = ["pi_ge", "half_gain"]

yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]] = np.reshape(yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]], (4,4))
yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]][qubit_i, qubit_i] = int(round(new_amp_half_ge))
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]][qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]][qubit_i, qubit_i]})
print(f'*New {yaml_update[0]}.{yaml_update[1]}\n', yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]])
yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]] = yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]].flatten().astype(int).tolist()
print(f'*New {yaml_update[0]}.{yaml_update[1]} flattened\n{yaml_cfg.device.qubit.pulses[yaml_update[0]][yaml_update[1]]}')
print(results_Qi)

fitparams [None, 0.017453292519943295]
From amps: adjust amplitude to 3595 / 1.0526538911009298 = 3415.1776100310894
From avgi: adjust amplitude to 3595 / 1.004050416624822 = 3580.4974934274874
From avgq: adjust amplitude to 3595 / 0.947346260195975 = 3794.8109904991993
*Set pi/2 pulse gain for qubit 3 to 3580*
*New pihalf_ge_robust.gain
 [[ 7569  7569  7569  7569]
 [ 4982  4982  4982  4982]
 [31856 31856 31856 31856]
 [ 3595  3595  3595  3580]]
*New pihalf_ge_robust.gain flattened
[7569, 7569, 7569, 7569, 4982, 4982, 4982, 4982, 31856, 31856, 31856, 31856, 3595, 3595, 3595, 3580]
{'Res freq': 6971.126666666667, 'Res freq (yaml)': -228.87333333333333, 'Res kappa': 0.21493746441942563, 'Res chi shift (e - g)': -0.8046357615893953, 'Qubit freq': 4383.7527021237, 'Pi/2 pulse gain': 3580}


In [ ]:
if save_data: npulsecalib_gehalf.save_data()

## (Divide gain) Calibrate $\pi/2$ pulse

### Amplitude rabi for $\pi/2$ pulse (divide gain) (use this for RB)

In [ ]:
divide_len = False

In [ ]:
pi_len = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_len', pi_len)

In [ ]:
amprabi_3 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_3.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(1.5*0.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_3.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_len, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss',
    checkEF=False, 
    n_pulses=1,
    test_pi_half=True,
    divide_len=divide_len,
    qTest=qubit_i,
    qZZ=None,
)
amprabi_3.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
# # TESTING
# # amprabi_3 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_3.data = temp_data

amprabi_3.analyze(fit=True)

amprabi_3_fit, amprabi_3_fit_err = meas.fitting.get_best_fit(amprabi_3.data)

p = amprabi_3_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_3.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_3.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ge.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ge.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma}')
print(results_Qi)

In [ ]:
if save_data: saved_files.append(amprabi_3.save_data())

### GE $\pi/2$ error amplification (divide gain)

In [ ]:
divide_len = False

In [ ]:
npulsecalib_gehalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptQ{qubit_i}",
    config_file=config_path,
)

npulsecalib_gehalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_gehalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=20,
    pulse_type='gauss',
    checkEF=False, 
    qTest=qubit_i,
    qZZ=None,
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,

    post_process='scale',
    singleshot_reps=15000,
)

print(npulsecalib_gehalf.cfg)
import Pyro4.util
try:
    npulsecalib_gehalf.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_gehalf.data
npulsecalib_gehalf.analyze(fit=True, scale=None)
npulsecalib_gehalf.display(fit=True, scale=None)

# npulsecalib_gehalf_fit, npulsecalib_gehalf_fit_err = meas.fitting.get_best_fit(npulsecalib_gehalf.data)
fit_axis = 'avgi'
npulsecalib_gehalf_fit = data['fit_avgi']
npulsecalib_gehalf_fit_err = data['fit_err_avgi']

angle_err = npulsecalib_gehalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ge = np.reshape(npulsecalib_gehalf.cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i] = int(round(new_amp_half_ge))
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ge.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ge.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma}')
print(results_Qi)

In [ ]:
if save_data: npulsecalib_gehalf.save_data()

## Resonator chi shift (g/e)

In [34]:
rspec_chi = meas.ResonatorSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"resonator_spectroscopy_chi_qubit{qubit_i}",
    config_file=config_path,
)
rspec_chi.cfg = AttrDict(deepcopy(yaml_cfg))

span = 3 # MHz
npts = 151
rspec_chi.cfg.expt = dict(
    start=rspec_chi.cfg.device.readout.frequency[qubit_i]-span/2, # MHz
    # start=rspec_chi.cfg.device.readout.frequency[qubit_i]-rspec_chi.cfg.device.readout.lo_sideband[qubit_i]*span, # MHz
    step=span/npts,
    expts=npts,
    reps=reps*rounds,
    pulse_e=True, # add ge pi pulse prior to measurement
    pulse_f=False, # add ef pi pulse prior to measurement
    qTest=qubit_i,
    qZZ=None,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

rspec_chi.cfg.device.readout.gain[qubit_i] /= 2
# rspec_chi.cfg.device.readout.readout_length[qubit_i] *= 2

# rspec_chi.cfg.device.readout.relax_delay = 100 # Wait time between experiments [us]
rspec_chi.go(analyze=False, display=False, progress=True, save=False)

  0%|          | 0/151 [00:00<?, ?it/s]

In [35]:
# # TESTING
# # rspec_chi = meas.ResonatorSpectroscopyExperiment(config_file=config_path,) 
# temp_data, attrs = prev_data(expt_path, '00001_resonator_spectroscopy_qubit2.h5')
# rspec_chi.data = temp_data

rspec_chi.data = rspec_chi.analyze(fit=True)
rspec_chi_fit = rspec_chi.data['fit']
if not isinstance(rspec_chi_fit, (list, np.ndarray)):
    rspec_chi.display(fit=False)
    assert False, 'Fit failed, please manually fix!'
rspec_chi.display(fit=True)
# res_freq_e = rspec_chi_fit[0] - yaml_cfg.hw.soc.dacs.readout.mixer_freq[qubit_i]
res_freq_e = rspec_chi.data['xpts'][np.argmin(rspec_chi.data['amps'])]
# res_freq_e = rspec_chi.data['xpts'][np.argmax(rspec_chi.data['amps'])]
print('Res freq e', res_freq_e)
res_freq_g = yaml_cfg.device.readout.frequency[qubit_i]
print('Res freq g', res_freq_g)
# res_freq_e = -83
# results_Qi.update({'Res chi shift (e - g)': yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_e - yaml_cfg.device.readout.frequency[qubit_i])})
chi_shift_eg = yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_e - res_freq_g)
print('Chi shift (e-g):', chi_shift_eg)
results_Qi.update({'Res chi shift (e - g)': chi_shift_eg})
# res_freq_e = 6953.5
# results_Qi.update({'Res chi shift (e - g)': res_freq_e - (5925 + yaml_cfg.device.readout.lo_sideband[qubit_i]*(yaml_cfg.hw.soc.dacs.readout.mixer_freq + yaml_cfg.device.readout.frequency[qubit_i]))})
print(f'qubit {qubit_i}', results_Qi)

electrical delay phase correction (no mux support) (deg) -0.8671127435910813
param guess [6971.090066225166, 34855.45033112583, 34855.45033112583, 0, -0.03265461156041281, 0.9251021987094618]
param bounds ([6969.619867549669, 0, 0, -6.283185307179586, -0.04898191734061922, -inf], [6972.560264900662, 1000000000.0, 100000000.0, 6.283185307179586, 0.04898191734061922, 1.0212547345778735])
popt [ 6.97032831e+03  7.24724538e+04  7.62323403e+04  9.33117308e-02
 -2.34562057e-02 -1.59017410e-02]

Freq with minimum transmission: -279.6649006622513
Freq with maximum transmission: -279.20794701986716
From fit:
	f0: 6970.328309213009
	Qi: 72472.45384249448 	 kappa_i/2pi: 0.09617900236083812
	Qe: 76232.34031360397 	 kappa_e/2pi: 0.09143531840342996
	Q0: 37152.43207884447
	kappa [MHz]: 0.18761432076426807
	phi [radians]: 0.09331173076857704
baseline 764.315295153289
Res freq e -279.64503311258244
Res freq g -278.89999999999964
Chi shift (e-g): -0.7450331125828029
qubit 3 {'Res chi shift (e - g)': -0

In [49]:
if save_data: saved_files.append(rspec_chi.save_data())

# Qubit ef spectroscopy

## Pulse Probe EF spectroscopy

In [161]:
qEFspec = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_EF_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qEFspec.cfg = AttrDict(deepcopy(yaml_cfg))

span = 50
npts = 250

qEFspec.cfg.expt = dict(
    start=np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))[qubit_i,qubit_i]-0.5*span, # resonator frequency to be mixed up [MHz]
    step=span/npts, # min step ~1 Hz
    expts=npts, # Number of experiments stepping from start
    reps=reps, # Number of averages per point
    rounds=rounds, # Number of start to finish sweeps to average over
    length=1, # ef probe constant pulse length [us]
    gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i]/50), # ef pulse gain
    # gain=5000,
    pulse_type='gauss',
    qTest=qubit_i,
    qZZ=None,
    checkEF=True,
)
# qEFspec.cfg.device.readout.relax_delay = qEFspec.cfg.device.qubit.T1[qubit_i] # Wait time between experiments [us]
qEFspec.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe EF on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [162]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = qEFspec.analyze(data=temp_data, fit=True)

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    try: 
        qEFspec.analyze(fit=True, signs=[1, sign, sign])
        fit_err_i = qEFspec.data['fit_err_avgi'][2][2]
        fit_err_q = qEFspec.data['fit_err_avgq'][2][2]
        if fit_err_i < best_fit_err_i:
            best_signs[1] = sign
            best_fit_err_i = fit_err_i
        if fit_err_q < best_fit_err_q:
            best_signs[2] = sign
            best_fit_err_q = fit_err_q
    except: continue
if best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qEFspec.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qEFspec.analyze(fit=True, signs=best_signs)
qEFspec.display(fit=True, signs=best_signs)

qEFspec_fit, qEFspec_fit_err = meas.fitting.get_best_fit(qEFspec.data)
yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = qEFspec_fit[2]
print(f'*Set qubit {qubit_i} f_ef to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 4216.157186239764, HWHM 1.979742709339923
Found peak in I at [MHz] 4202.3042692789695, HWHM 12.275497056540667
Found peak in Q at [MHz] 4216.073677951402, HWHM 1.7793481167801906
best 2 0.07668230939675232
*Set qubit 3 f_ef to 4216.073677951402*
{'Qubit EF freq': 4216.073677951402, 'T2R EF': 1.039341517857143, 'T2R EF err': 0.30606294265120315}


In [107]:
if save_data: saved_files.append(qEFspec.save_data())

## Pulse Probe EF spectroscopy round 2

In [199]:
qEFspec2 = meas.PulseProbeSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"qubit_EF_spectroscopy_qubit{qubit_i}",
    config_file=config_path,
)
qEFspec2.cfg = AttrDict(deepcopy(yaml_cfg))

span = 15
npts = 250

qEFspec2.cfg.expt = dict(
    start=np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))[qubit_i,qubit_i]-0.5*span, # resonator frequency to be mixed up [MHz]
    step=span/npts, # min step ~1 Hz
    expts=npts, # Number of experiments stepping from start
    reps=reps, # Number of averages per point
    rounds=rounds, # Number of start to finish sweeps to average over
    length=1, # ef probe constant pulse length [us]
    gain=int(np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i]/80), # ef pulse gain
    pulse_type='gauss',
    qTest=qubit_i,
    qZZ=None,
    checkEF=True,
)
qEFspec2.go(analyze=False, display=False, progress=True, save=False)

Running pulse probe EF on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

In [200]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = qEFspec2.analyze(data=temp_data, fit=True)

best_signs = [1, 0, 0]
best_fit_err_i = np.inf
best_fit_err_q = np.inf
for sign in [1, -1]:
    qEFspec2.analyze(fit=True, signs=[1, sign, sign])
    fit_err_i = qEFspec2.data['fit_err_avgi'][2][2]
    fit_err_q = qEFspec2.data['fit_err_avgq'][2][2]
    if fit_err_i < best_fit_err_i:
        best_signs[1] = sign
        best_fit_err_i = fit_err_i
    if fit_err_q < best_fit_err_q:
        best_signs[2] = sign
        best_fit_err_q = fit_err_q
    # except: continue
if best_fit_err_i == np.inf and best_fit_err_q == np.inf:
    qEFspec2.display(fit=False)
    assert False, 'Fit failed, please manually fix!'

qEFspec2.analyze(fit=True, signs=best_signs)
qEFspec2.display(fit=True, signs=best_signs)

qEFspec2_fit, qEFspec2_fit_err = meas.fitting.get_best_fit(qEFspec2.data)
yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = qEFspec2_fit[2]
qubit_hwhm = qEFspec2_fit[3]
print(f'*Set qubit {qubit_i} f_ef to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(results_Qi)

Found peak in amps at [MHz] 4205.916183716192, HWHM 1.0281994643793693
Found peak in I at [MHz] 4205.9278169298605, HWHM 1.13050923098174
Found peak in Q at [MHz] 4205.893765280253, HWHM 1.0988053354217038
best 2 0.031104358630070213
*Set qubit 3 f_ef to 4205.893765280253*
{'Qubit EF freq': 4205.893765280253, 'T2R EF': 1.039341517857143, 'T2R EF err': 0.2842483864775508, 'Pi/2 EF pulse gain': 3650, 'T1': 55.69833035027093, 'T1 err': 2.013371053265863}


In [110]:
if save_data: saved_files.append(qEFspec2.save_data())

## Amplitude EF Rabi

The EF pi pulse amplitude should be stored in <code>cfg.device.qubit.pulses.pi_ef.gain</code>.

In [186]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_lenEF = 0.025
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

qubit 3 pi_lenEF 0.03


In [17]:
amprabiEF = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="amp_rabi_EF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

amprabiEF.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 101

amprabiEF.cfg.expt = dict(
    start=1, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps*rounds,
    rounds=1,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - default from cfg [us]
    pulse_type='gauss',
    checkEF=True,
    qTest=qubit_i,
    qZZ=None,

    readout_cool=False,
    # n_init_readout=1,
    # n_trig=2,
    # init_read_wait_us=5,
    # singleshot_reps=20000,
    # use_gf_readout=True,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    
)

# print(amprabiEF.cfg)
amprabiEF.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi EF on Q1 


  0%|          | 0/101000 [00:00<?, ?it/s]

(4, 101, 1000)


In [18]:
# TESTING
# amprabiEF = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_EF_qubit1.h5')
# amprabiEF.data = temp_data

# amprabiEF.analyze(fit=False)
# amprabiEF.display(fit=False)

amprabiEF.analyze(fit=True, post_select=False)

amprabiEF_fit, amprabiEF_fit_err = meas.fitting.get_best_fit(amprabiEF.data)

p = amprabiEF_fit

if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabiEF.display(fit=False)
    print('WARNING: Fit pi pulse gain is long, please double check!')
    pi_gain = 30000
else: amprabiEF.display(fit=True)


best 0 0.04915627151792078
Pi gain from amps data [dac units]: 3332
	Pi/2 gain from amps data [dac units]: 1666
Pi gain from avgi data [dac units]: 5356
	Pi/2 gain from avgi data [dac units]: 2678
Pi gain from avgq data [dac units]: 3471
	Pi/2 gain from avgq data [dac units]: 1735


In [19]:
print(f'*Set EF pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ef.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set EF pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]})
print(f'*New pi_ef.gain\n', yaml_cfg.device.qubit.pulses.pi_ef.gain)
yaml_cfg.device.qubit.pulses.pi_ef.gain = yaml_cfg.device.qubit.pulses.pi_ef.gain.flatten().astype(int).tolist()
print(f'*New pi_ef.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(results_Qi)

*Set EF pi pulse sigma for qubit 1 to 0.025*
*Set EF pi pulse gain for qubit 1 to 3332*
*New pi_ef.gain
 [[ 1280     0     0     0]
 [    0  3332     0     0]
 [10000 10079 10015     0]
 [  300   300     0   300]]
*New pi_ef.gain flattened
[1280, 0, 0, 0, 0, 3332, 0, 0, 10000, 10079, 10015, 0, 300, 300, 0, 300]
{'Pi EF pulse gain': 3332}


In [115]:
if save_data: saved_files.append(amprabiEF.save_data())

## Length EF Rabi

In [ ]:
lengthrabiEF = meas.LengthRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"length_rabiEF_qubit{qubit_i}",
    config_file=config_path,
)
lengthrabiEF.cfg = AttrDict(deepcopy(yaml_cfg))

lengthrabiEF.cfg.expt = dict(
    start=soc.cycles2us(0), # pulse length (sigma) [us]
    step=soc.cycles2us(1), # [us]
    expts=121,
    reps=reps*rounds,
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,
)
# lengthrabiEF.cfg.device.readout.relax_delay = 50 # Wait time between experiments [us]

import Pyro4.util
try:
    lengthrabiEF.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# # lengthrabiEF = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# lengthrabiEF.data = temp_data

lengthrabiEF.analyze(fit=True, fit_func='sin')

lengthrabiEF_fit, lengthrabiEF_fit_err = meas.fitting.get_best_fit(lengthrabiEF.data) #, fitfunc=fitter.decaysin)

lengthrabiEF.display(fit=False, fit_func='sin')
# p = lengthrabiEF_fit
# if p[2] > 180: p[2] = p[2] - 360
# elif p[2] < -180: p[2] = p[2] + 360
# if p[2] < 0: pi_lenEF = (1/2 - p[2]/180)/2/p[1]
# else: pi_lenEF = (3/2 - p[2]/180)/2/p[1]
# pi_lenEF = np.max((0.9*pi_lenEF, 0.015)) # since we run the length rabi at a < max gain

# lengthrabiEF_decay = p[3]

# print(f'Will use sigma {pi_lenEF} for amplitude rabi EF')

In [ ]:
if save_data: saved_files.append(lengthrabiEF.save_data())

## Qubit temperature

In [33]:
amprabiEF2 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="amp_rabi_EF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

amprabiEF2.cfg = AttrDict(deepcopy(yaml_cfg))

reps_temp=500
rounds_temp=reps*rounds*100//500

span = np.reshape(amprabiEF2.cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i]
npts = 4
# reps_temp = 10
# rounds_temp = 10
print('reps', reps_temp, 'rounds', rounds_temp)

amprabiEF2.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts+1,
    reps=reps_temp*rounds_temp,
    rounds=1,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - default from cfg [us]
    pulse_ge=False, # set to false when measuring qubit temp
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,


    # readout_cool=False,
    readout_cool=True,
    n_init_readout=1,
    n_trig=1,
    init_read_wait_us=2,
    singleshot_reps=10000,
    
    # use_gf_readout=True,
    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# print(amprabiEF2.cfg)
amprabiEF2.go(analyze=False, display=False, progress=True, save=False)

reps 500 rounds 200
Running amp rabi EF on Q1 


  0%|          | 0/2 [00:00<?, ?it/s]

thresholds [0, 327.10455367508064, 0, 0]
angles [0, 0.0, 0, 0]


  0%|          | 0/500000 [00:00<?, ?it/s]

(4, 5, 100000)


In [30]:
post_select = False
# post_select = True
ps_threshold = None
# ps_threshold = 450

In [34]:
# TESTING
# amprabiEF_test = meas.AmplitudeRabiExperiment(config_file=config_path,)
# amprabiEF2 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00004_amp_rabi_EF_qubit1.h5')
# amprabiEF2.data = temp_data

# amprabiEF2.analyze(fit=False)
# amprabiEF2.display(fit=False)

# fitparams = [None]*4
# fitparams[1] = 2/amprabiEF2.data['xpts'][-1]

# amprabiEF2_fit, amprabiEF2_fit_err = meas.fitting.get_best_fit(amprabiEF2.data)

# NATURAL CONSTANTS
h = 6.62607015e-34
fge = 1e6*(np.reshape(amprabiEF2.cfg.device.qubit.f_ge, (4,4))[qubit_i, qubit_i] + amprabiEF2.cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i])
kB = 1.380649e-23
################

fit_axis = 'amps'
# fit_axis = 'avgi' if post_select else 'amps'

amprabiEF.analyze(fit=False, post_select=post_select, ps_threshold=ps_threshold)
amprabiEF_data = amprabiEF.data[fit_axis]
# amprabiEF2_data = (amprabiEF2.data[fit_axis] + temp_data[fit_axis])/2
# p = amprabiEF.data[f'fit_{fit_axis}']
# efamp_normal = p[0]
efamp_normal = (max(amprabiEF.data[fit_axis]) - min(amprabiEF.data[fit_axis]))

amprabiEF2.analyze(fit=False, post_select=post_select, ps_threshold=ps_threshold)
amprabiEF2_data = amprabiEF2.data[fit_axis]
# p = amprabiEF2.data[f'fit_{fit_axis}']
# efamp_nopulse = p[0]
# fit_vals = meas.fitting.sinfunc(amprabiEF.data['xpts'], *p)
# std_dev_no_pulse = np.std(fit_vals[:10])
# efamp_nopulse = (max(fit_vals) - min(fit_vals))-2*std_dev_no_pulse
efamp_nopulse = (max(amprabiEF2.data[fit_axis]) - min(amprabiEF2.data[fit_axis]))

# amprabiEF2.display(fit=True, fit_func='sin')

print('yscale normal:', efamp_normal)
print('yscale no pulse:', efamp_nopulse)

temp = 1e3*-h*fge/(kB*np.log(efamp_nopulse/(efamp_normal+efamp_nopulse)))

print(f'qubit {qubit_i}')
print('Qubit temp [mK]:', temp)
results_Qi.update({'Qubit temp [mK]':temp})
print('Qubit E thermal population (%):', efamp_nopulse/efamp_normal*100)

amprabiEF2.display(fit=False)
results_Qi.update({'Qubit E thermal population (%)':efamp_nopulse/efamp_normal*100})
print(results_Qi)

IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [64]:
if save_data: saved_files.append(amprabiEF2.save_data())

## EF Ramsey Fringe
Update the ef frequency in <code>cfg.qubit.f_ef</code>.

In [189]:
qubitEF_hwhm = 2
# short_T1 = False

In [208]:
ramseyEF_freq = max((min((np.abs(np.ceil(qubitEF_hwhm)), 3)), 2))

t2EFr = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="ramseyEF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

t2EFr.cfg = AttrDict(deepcopy(yaml_cfg))

t2EFr.cfg.expt = dict(
    start=0, # wait time tau [us]
    # step=soc.cycles2us(10), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    step=soc.cycles2us(2), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    # step=0.05, # [us]
    expts=151,
    # ramsey_freq=ramseyEF_freq, # [MHz]
    ramsey_freq=5.0, # [MHz]
    checkEF=True,
    reps=reps*rounds,
    rounds=1,
    qTest=qubit_i,
    qZZ=None,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=use_robust_pulses,
)

try:
    t2EFr.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

Running Ramsey EF on Q3 


  0%|          | 0/151000 [00:00<?, ?it/s]

In [209]:
# # TESTING
# # t2EFr = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_ramsey_qubit2.h5')
# t2EFr.data = temp_data

t2EFr.analyze(fit=True)
t2EFr_fit, t2EFr_fit_err, t2EFr_adjust = meas.fitting.get_best_fit(t2EFr.data, get_best_data_params=['f_adjust_ramsey'])
old_qubit_freq = np.reshape(t2EFr.cfg.device.qubit.f_ef, (4,4))[qubit_i, qubit_i]

if not isinstance(t2EFr_fit, (list, np.ndarray)):
    t2EFr.display(fit=False)
    assert False, 'Fit failed, please manually fix!'
t2EFr.display(fit=True)
yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = old_qubit_freq + t2EFr_adjust[0]
alt_qubit_freq = old_qubit_freq + t2EFr_adjust[1]
print(f'*Set qubit {qubit_i} EF freq to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*New f_ef\n', yaml_cfg.device.qubit.f_ef)
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(f'*New f_ef flattened\n{yaml_cfg.device.qubit.f_ef}')
results_Qi.update({'T2R EF':t2EFr_fit[3], 'T2R EF err':np.sqrt(t2EFr_fit_err[3][3])})
print(all_results)

best 1 0.12135550201556408
Current pi pulse frequency: 4205.885958110913
Fit frequency from amps [MHz]: 5.312776082975846 +/- 0.19097927577666232
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4205.573182027937
 	4195.573182027937
T2 Ramsey from fit amps [us]: 0.2078683035714286 +/- 0.05900615911800343
Current pi pulse frequency: 4205.885958110913
Fit frequency from I [MHz]: 5.3276007029970005 +/- 0.19497938787625346
Suggested new pi pulse frequency from fit I [MHz]:
 	4205.5583574079155
 	4195.5583574079155
T2 Ramsey from fit I [us]: 0.2078683035714286 +/- 0.05902113089483773
Fit frequency from Q [MHz]: 5.53030021858428 +/- 0.21194802629833834
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4205.355657892328
 	4195.355657892328
T2 Ramsey from fit Q [us]: 0.2078683035714286 +/- 0.06573980089549061
*Set qubit 3 EF freq to 4205.5583574079155*
*New f_ef
 [[3884.96686009    0.            0.            0.        ]
 [   0.         3347.84622574    0.            0.        ]


In [64]:
if save_data: saved_files.append(t2EFr.save_data())

## Amplitude EF Rabi round 2

In [78]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_lenEF = 0.030
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

qubit 3 pi_lenEF 0.03


In [ ]:
amprabiEF = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="amp_rabi_EF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

amprabiEF.cfg = AttrDict(deepcopy(yaml_cfg))

span = min((32000, int(2*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 101

amprabiEF.cfg.expt = dict(
    start=1, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    rounds=rounds,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - default from cfg [us]
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,
    resonator_reset=resonator_reset,
    full_mux_expt=full_mux_expt,
)

# print(amprabiEF.cfg)
amprabiEF.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi EF on Q3 


  0%|          | 0/10 [00:00<?, ?it/s]

(4, 101, 100)


In [100]:
# TESTING
# amprabiEF = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_EF_qubit1.h5')
# amprabiEF.data = temp_data

# amprabiEF.analyze(fit=False)
# amprabiEF.display(fit=False)

amprabiEF.analyze(fit=True)

amprabiEF_fit, amprabiEF_fit_err = meas.fitting.get_best_fit(amprabiEF.data)

p = amprabiEF_fit

if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabiEF.display(fit=False)
    print('WARNING: Fit pi pulse gain is long, please double check!')
    pi_gain = 30000
else: amprabiEF.display(fit=True)

print(f'*Set EF pi pulse sigma for qubit {qubit_i} to {np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]}*')
yaml_cfg.device.qubit.pulses.pi_ef.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set EF pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]})
print(f'*New pi_ef.gain\n', yaml_cfg.device.qubit.pulses.pi_ef.gain)
yaml_cfg.device.qubit.pulses.pi_ef.gain = yaml_cfg.device.qubit.pulses.pi_ef.gain.flatten().astype(int).tolist()
print(f'*New pi_ef.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(results_Qi)

best 0 0.05280407148696756
Pi gain from amps data [dac units]: 1093
	Pi/2 gain from amps data [dac units]: 546
Pi gain from avgi data [dac units]: 4011
	Pi/2 gain from avgi data [dac units]: 2005
Pi gain from avgq data [dac units]: 363
	Pi/2 gain from avgq data [dac units]: 181
*Set EF pi pulse sigma for qubit 3 to 0.03*
*Set EF pi pulse gain for qubit 3 to 1093*
*New pi_ef.gain
 [[1280    0    0    0]
 [   0 1912    0    0]
 [   0    0 6380    0]
 [ 300  300    0 1093]]
*New pi_ef.gain flattened
[1280, 0, 0, 0, 0, 1912, 0, 0, 0, 0, 6380, 0, 300, 300, 0, 1093]
{'Res freq': 6971.086666666667, 'Res freq (yaml)': -278.91333333333296, 'Res kappa': 0.19749745591104786, 'Qubit freq': 4381.201533272392, 'Pi pulse gain': 1147, 'T1': 56.17688427668649, 'T1 err': 2.67362068432995, 'T2R': 53.42815301037524, 'T2R err': 7.869086115111744, 'Qubit EF freq': 4381.19346042819, 'T2R EF': 3.7111881566221205, 'T2R EF err': 0.9227106213738326, 'Pi EF pulse gain': 1093}


In [ ]:
if save_data: saved_files.append(amprabiEF.save_data())

## EF Ramsey Fringe Round 2
Update the T2 EF but not the frequency

In [24]:
# yaml_cfg.device.qubit.f_ge[qubit_i] = 145.572034953271
# qubitEF_hwhm = 7
# short_T1 = False

In [25]:
t2EFr_2 = meas.RamseyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix="ramseyEF"+f"_qubit{qubit_i}",
    config_file=config_path,
)

t2EFr_2.cfg = AttrDict(deepcopy(yaml_cfg))

t2EFr_2.cfg.expt = dict(
    start=0, # wait time tau [us]
    step=soc.cycles2us(80), # [us] make sure nyquist freq = 0.5 * (1/step) > ramsey (signal) freq!
    # step=0.05, # [us]
    expts=151,
    ramsey_freq=ramseyEF_freq/10, # [MHz]
    checkEF=True,
    reps=reps,
    rounds=rounds,
    qTest=qubit_i,
    qZZ=None,
)

try:
    t2EFr_2.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

Running Ramsey EF on Q2 


  0%|          | 0/10 [00:00<?, ?it/s]

In [26]:
# # TESTING
# # t2EFr_2 = meas.RamseyExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00000_ramsey_qubit2.h5')
# t2EFr_2.data = temp_data

t2EFr_2.analyze(fit=True)
t2EFr_2_fit, t2EFr_2_fit_err, t2EFr_2_adjust = meas.fitting.get_best_fit(t2EFr_2.data, get_best_data_params=['f_adjust_ramsey'])
old_qubit_freq = np.reshape(t2EFr_2.cfg.device.qubit.f_ef, (4,4))[qubit_i, qubit_i]

if not isinstance(t2EFr_2_fit, (list, np.ndarray)):
    t2EFr_2.display(fit=False)
    assert False, 'Fit failed, please manually fix!'
t2EFr_2.display(fit=True)

if t2EFr_2_adjust[0] > ramseyEF_freq/10:
    print('WARNING: your EF frequency is not converging!')
else:
    # yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
    # yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = old_qubit_freq + t2EFr_2_adjust[0]
    # print(f'*Set qubit {qubit_i} EF freq to {yaml_cfg.device.qubit.f_ef[qubit_i]}*')
    # print(f'*New f_ef\n', yaml_cfg.device.qubit.f_ef)
    # yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
    # print(f'*New f_ef flattened\n{yaml_cfg.device.qubit.f_ef}')
    results_Qi.update({'T2R EF':t2EFr_2_fit[3], 'T2R EF err':np.sqrt(t2EFr_2_fit_err[3][3])})
    # results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
    print(results_Qi)

best 1 0.036576327133675327
Current pi pulse frequency: 4572.9717022650675
Fit frequency from amps [MHz]: 0.20741349443690102 +/- 0.0008558715355503085
Suggested new pi pulse frequencies from fit amps [MHz]:
 	4572.96428877063
 	4572.564288770631
T2 Ramsey from fit amps [us]: 30.454110313987638 +/- 4.8596737393755145
Current pi pulse frequency: 4572.9717022650675
Fit frequency from I [MHz]: 0.20680278225332982 +/- 0.00040793370161809264
Suggested new pi pulse frequency from fit I [MHz]:
 	4572.964899482814
 	4572.564899482814
T2 Ramsey from fit I [us]: 27.060264891228364 +/- 1.8466041530100628
Fit frequency from Q [MHz]: 0.2 +/- inf
Suggested new pi pulse frequencies from fit Q [MHz]:
 	4572.9717022650675
 	4572.571702265068
T2 Ramsey from fit Q [us]: 27.71577380952381 +/- inf
{'T2R EF': 27.060264891228364, 'T2R EF err': 1.8466041530100628}


In [ ]:
if save_data: saved_files.append(t2EFr.save_data())

## EF error amplification

In [94]:
npulsecalibEF = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptEFQ{qubit_i}",
    config_file=config_path,
)

npulsecalibEF.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalibEF.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=10,
    pulse_type='gauss',
    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,
    test_pi_half=False,
    error_amp=True,
    resonator_reset=resonator_reset,
    full_mux_expt=full_mux_expt,

    post_process='scale',
    singleshot_reps=15000,
    # singleshot_reps=1000,
)

print(npulsecalibEF.cfg)
try:
    npulsecalibEF.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-439.97500000000036, 0.0, 452.3682333333336, -278.91333333333296], 'gain': [0.17, 0.025, 0.25, 0.17], 'kappa_ext': [0.157, 0.065, 0.182, 0.104], 'kerr': [-0.0045, 0, 0.008, -0.003], 't_rise_reset': [0.5, 1, 0.5, 0.5], 'readout_length': [4, 4, 4, 4], 'frequency_ef': [-441.91408404503545, -1.5099337748344372, 450.44439227373095, -280.30732891832196], 'gain_ef': [0.001, 0.025, 0.25, 0.17], 'readout_length_ef': [4, 4, 4, 4], 'relax_delay': [450, 450, 450, 225.0]}, 'qubit': {'f_ge': [4111.144520031796, 4105.783958808882, 4111.0028964412795, 4110.5966963096635, 3442.9795552267146, 3448.3482722012286, 3447.233490336077, 3447.0285717924894, 4762.767009306428, 4761.793655204115, 4762.900421466317, 4762.367822139689, 4380.662337410765, 4379.890920412824, 4380.6668186549305, 4381.201533272392], 'f_ef': [3885.3117884920853, 0.0, 0.0, 0.0, 0.0, 3347.8480888529375, 0.0, 0.0, 4571.9056482340775, 4571.070566218902, 

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (3)
g unrotated averages:
Amps 474.855206456419 +/- 385.44830677901336
e unrotated averages:
Amps 1254.0064285520539 +/- 442.2000705777346
ge fidelity (%): 72.42574438295885 	 angle (deg): 0.0 	 threshold ge: 759.4211635666827


  0%|          | 0/10 [00:00<?, ?it/s]

 s:\Connie\experiments\qram_tprocv1_expts\experiments\clifford_averager_program.py: 1877

In [95]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalibEF.data
npulsecalibEF.analyze(fit=True)
npulsecalibEF.display(fit=True)

# npulsecalib_fit, npulsecalib_fit_err = meas.fitting.get_best_fit(npulsecalibEF.data)
fit_axis = 'avgi'
npulsecalibEF_fit = data[f'fit_{fit_axis}']
npulsecalibEF_fit_err = data[f'fit_err_{fit_axis}']

angle_err = npulsecalibEF_fit[1]
amp_ratio = (180 - angle_err) / 180
new_amp_ef = np.reshape(npulsecalibEF.cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ef.gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i] = int(round(new_amp_ef))
print(f'*Set EF pi pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.gain[qubit_i, qubit_i]})
print(f'*New pi_ef.gain\n', yaml_cfg.device.qubit.pulses.pi_ef.gain)
yaml_cfg.device.qubit.pulses.pi_ef.gain = yaml_cfg.device.qubit.pulses.pi_ef.gain.flatten().tolist()
print(f'*New pi_ef.gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(results_Qi)

fitparams [None, 0.017453292519943295]
Attempted to init fitparam 0 to 0.0, which is out of bounds 0.0 to 0.0. Instead init to 0.0
From amps: adjust amplitude to 320 / 0.9978554230773876 = 320.68773952555125
From avgi: adjust amplitude to 320 / 0.9978554230773876 = 320.68773952555125
From avgq: adjust amplitude to 320 / 0.999903037263778 = 320.03103108444986
*Set EF pi pulse gain for qubit 3 to 1095*
*New pi_ef.gain
 [[1280    0    0    0]
 [   0 1912    0    0]
 [   0    0 6380    0]
 [ 300  300    0 1095]]
*New pi_ef.gain flattened
[1280, 0, 0, 0, 0, 1912, 0, 0, 0, 0, 6380, 0, 300, 300, 0, 1095]
{'Res freq': 6971.086666666667, 'Res freq (yaml)': -278.91333333333296, 'Res kappa': 0.19749745591104786, 'Qubit freq': 4381.201533272392, 'Pi pulse gain': 1147, 'T1': 56.17688427668649, 'T1 err': 2.67362068432995, 'T2R': 53.42815301037524, 'T2R err': 7.869086115111744, 'Qubit EF freq': 4381.19346042819, 'T2R EF': 23.803256694009963, 'T2R EF err': 6.133913126035808, 'Pi EF pulse gain': 1095}


In [ ]:
if save_data: npulsecalibEF.save_data()

## (Divide len) Calibrate EF $\pi/2$ pulse
this is mostly only for RB

### Amplitude rabi for EF $\pi/2$ pulse (divide len)

In [204]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

qubit 3 pi_lenEF 0.03


In [205]:
divide_len = True

In [206]:
amprabi_4 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_4.cfg = AttrDict(deepcopy(yaml_cfg))

n_pulses = 1
span = min((32000, int(1.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_4.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss' if not use_robust_pulses else 'robust', # EF pulse will always be gauss, need this because will throw an error otherwise even if only using robust for the g
    n_pulses=n_pulses,
    test_pi_half=True,
    divide_len=divide_len,
    qTest=qubit_i,
    qZZ=None,
    checkEF=True, 

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=use_robust_pulses,
)
amprabi_4.go(analyze=False, display=False, progress=True, save=False)

Running amp rabi EF on Q3 
Calibrating robust pulse


  0%|          | 0/10 [00:00<?, ?it/s]

(4, 151, 100)


In [207]:
# # TESTING
# # amprabi_4 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_4.data = temp_data

amprabi_4.analyze(fit=True)

amprabi_4_fit, amprabi_4_fit_err = meas.fitting.get_best_fit(amprabi_4.data)

p = amprabi_4_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_4.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_4.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ef.half_gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set pi/2 EF pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain = yaml_cfg.device.qubit.pulses.pi_ef.half_gain.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain}')
print(results_Qi)

best 1 0.18129534648810813
Pi gain from amps data [dac units]: 3276
	Pi/2 gain from amps data [dac units]: 1638
Pi gain from avgi data [dac units]: 3329
	Pi/2 gain from avgi data [dac units]: 1664
Pi gain from avgq data [dac units]: 3410
	Pi/2 gain from avgq data [dac units]: 1705
*Set pi/2 EF pulse gain for qubit 3 to 3329*
*New pi_ef.half_gain
 [[1243    0    0    0]
 [   0 1945    0    0]
 [   0 6472 6401    0]
 [1427 1601    0 3329]]
*New pi_ef.half_gain flattened
[1243, 0, 0, 0, 0, 1945, 0, 0, 0, 6472, 6401, 0, 1427, 1601, 0, 3329]
{'Qubit EF freq': 4205.885958110913, 'T2R EF': 1.039341517857143, 'T2R EF err': 0.3388830542258965, 'Pi/2 EF pulse gain': 3329, 'T1': 55.69833035027093, 'T1 err': 2.013371053265863}


In [ ]:
if save_data: saved_files.append(amprabi_4.save_data())

### EF $\pi/2$, -$\pi/2$ experiment

In [19]:
piminuspi_EF = meas.PiMinusPiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"PiMinusPi_EFExpt{qubit_i}",
    config_file=config_path,
)

span_f = 2.0
npts_f = 35
# npts_f = 50

# span_f = 0.0
# npts_f = 1

piminuspi_EF.cfg = AttrDict(deepcopy(yaml_cfg))
piminuspi_EF.cfg.expt = dict(
    start_N=0, # number gate repetitions
    # step_N=5,
    # step_N=2,
    # expts_N=15,
    step_N=4,
    expts_N=10,
    start_f=piminuspi_EF.cfg.device.qubit.f_ef[qubit_i*4 + qubit_i]-span_f/2,
    step_f=span_f/npts_f,
    expts_f=npts_f+1,
    reps=int(0.75*reps*rounds),
    loops=1,
    # reps=int(2*reps*rounds),
    # loops=5,
    pulse_type='gauss',

    checkEF=True,
    qTest=qubit_i,
    qZZ=None,

    # use_Y=True,

    test_pi_half=True,
    divide_len=divide_len,

    post_process='threshold',
    singleshot_reps=15000,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

print(piminuspi_EF.cfg)
try:
    piminuspi_EF.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-439.97500000000036, 0.0, 452.3682333333336, -278.89999999999964], 'gain': [0.1, 0.025, 0.25, 0.17], 'kappa_ext': [0.157, 0.065, 0.182, 0.104], 'kerr': [-0.0045, 0, 0.008, -0.003], 't_rise_reset': [0.5, 1, 0.5, 0.5], 'readout_length': [4, 4, 4, 4], 'frequency_ef': [-439.97500000000036, 0.0, 450.44439227373095, -280.30732891832196], 'readout_length_ef': 4, 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.141143385073, 4105.786145933962, 4110.958153299016, 4110.546407850519, 3442.9770233041277, 3448.328289144873, 3447.060317317807, 3446.4929792801067, 4762.761504132457, 4761.823567235975, 4762.898928901032, 4762.369813422167, 4380.698604555338, 4379.604868123916, 4380.717837106693, 4381.175711440575], 'f_ef': [3884.6269161242526, 0.0, 0.0, 0.0, 0.0, 3348.085433174199, 0.0, 0.0, 4571.9056482340775, 4570.996526541673, 4573.168610947125, 0.0, 4211.566315930455, 4205.61697813965, 0.0, 4209.925

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (3)
ge fidelity (%): 74.23333333333333 	 angle (deg): 0.0 	 threshold ge: 764.3224091008224


  0%|          | 0/10 [00:00<?, ?it/s]

: 

In [143]:
# # TESTING
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00000_PiMinusPi_EFExpt0.h5')
# piminuspi_EF.data =  temp_data
# piminuspi_EF.cfg = AttrDict(attrs['config'])


data = piminuspi_EF.data

# piminuspi_EF.analyze(fit=False)
# plt.figure()
# plt.plot(data['cycle_sweep'], data['avgi'][:,0], '.-')
# plt.xlabel('N Cycles $(\pi/2, -\pi/2)^N$')
# plt.ylabel('Avg I')
# plt.show()

piminuspi_EF.analyze(fit=True)
piminuspi_EF.display(fit=True)

yaml_cfg.device.qubit.f_ef = np.reshape(yaml_cfg.device.qubit.f_ef, (4,4))
yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] = data["best_freq"]
results_Qi.update({'Qubit EF freq':yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i] + yaml_cfg.hw.soc.dacs.qubit.mixer_freq[qubit_i]})
print(f'*Set qubit {qubit_i} EF freq to {yaml_cfg.device.qubit.f_ef[qubit_i, qubit_i]}*')
print(results_Qi)
print(f'*New f_ef\n', yaml_cfg.device.qubit.f_ef)
yaml_cfg.device.qubit.f_ef = yaml_cfg.device.qubit.f_ef.flatten().tolist()
print(f'*New f_ef flattened\n{yaml_cfg.device.qubit.f_ef}')

Fit best freq 4573.168610947125 which is 0.08048936338036583 away from old freq 4573.088121583744
*Set qubit 2 EF freq to 4573.168610947125*
{'Res freq': 7702.328233333334, 'Res freq (yaml)': 452.32823333333357, 'Res kappa': 0.2819786993431654, 'Res chi shift (e - g)': -0.943708609271539, 'Res chi shift (f - g)': -1.9172185430463742, 'Res freq ef (yaml)': 450.4510147902872, 'Pi/2 EF pulse gain': 6385, 'Qubit EF freq': 4573.168610947125}
*New f_ef
 [[3884.62691612    0.            0.            0.        ]
 [   0.         3348.08543317    0.            0.        ]
 [4571.90564823 4570.99652654 4573.16861095    0.        ]
 [4211.61773253 4205.61697814    0.         4209.92508626]]
*New f_ef flattened
[3884.6269161242526, 0.0, 0.0, 0.0, 0.0, 3348.085433174199, 0.0, 0.0, 4571.9056482340775, 4570.996526541673, 4573.168610947125, 0.0, 4211.617732525247, 4205.61697813965, 0.0, 4209.925086260656]


In [ ]:
if save_data: piminuspi.save_data()

### EF $\pi/2$ error amplification (divide len)

In [115]:
divide_len = True

In [138]:
npulsecalib_efhalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptEF_half_Q{qubit_i}",
    config_file=config_path,
)

npulsecalib_efhalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_efhalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    # loops=20,
    loops=5,
    pulse_type='gauss',
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,

    checkEF=True,
    qTest=qubit_i,
    qZZ=None,

    # readout_ge=False, # whether to readout at the g/e set point (True) or e/f set point (False)
    readout_ge=True, # whether to readout at the g/e set point (True) or e/f set point (False)

    post_process='scale',
    # post_process=None,
    singleshot_reps=15000,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,

    use_robust_pulses=True,
)

print(npulsecalib_efhalf.cfg)
import Pyro4.util
try:
    npulsecalib_efhalf.acquire(progress=True, debug=True)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-439.97500000000036, 0.0, 452.3682333333336, -278.89999999999964], 'gain': [0.17, 0.035, 0.25, 0.17], 'kappa_ext': [0.157, 0.065, 0.182, 0.104], 'kerr': [-0.0045, 0, 0.008, -0.003], 't_rise_reset': [0.5, 1, 0.5, 0.5], 'readout_length': [4, 4, 4, 4], 'frequency_ef': [-439.97500000000036, 0.0, 450.44439227373095, -280.30732891832196], 'readout_length_ef': 4, 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.097866562327, 4105.838904759637, 4110.958153299016, 4110.546407850519, 3442.8755721075536, 3448.5530242338505, 3447.060317317807, 3446.4929792801067, 4762.761504132457, 4761.823567235975, 4762.909473544484, 4762.369813422167, 4380.698604555338, 4379.604868123916, 4380.717837106693, 4381.248121020163], 'f_ge_robust': [4114.313815659153, 4108.147701523904, 4114.310372455052, 4114.310372455052, 3443.0032303441935, 3448.345849629168, 0, 0, 4763.941, 4763.941, 4763.941, 4763.941, 4384.2503422

  0%|          | 0/2 [00:00<?, ?it/s]

Qubit  (2)
g unrotated averages:
Amps 118.20467411805085 +/- 57.38787388991945
e unrotated averages:
Amps 279.91002155643014 +/- 67.09894931806625
ge fidelity (%): 81.78613640909393 	 angle (deg): 0.0 	 threshold ge: 196.18055470399742
thresholds=[0, 0, 196.18055470399742, 0],
angles=[0, 0, 0.0, 0],
ge_avgs=[array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), array([118.20467412,   0.        , 279.91002156,   0.        ]), array([0., 0., 0., 0.])],
counts_calib=[[13720.0, 1280.0], [1456.0, 13544.0]]


  0%|          | 0/5 [00:00<?, ?it/s]

 s:\Connie\experiments\qram_tprocv1_expts\experiments\clifford_averager_program.py: 1822

In [144]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_efhalf.data
npulsecalib_efhalf.analyze(fit=True, scale=None)
npulsecalib_efhalf.display(fit=True, scale=None)

# npulsecalib_efhalf_fit, npulsecalib_efhalf_fit_err = meas.fitting.get_best_fit(npulsecalib_efhalf.data)
fit_axis = 'avgi'
npulsecalib_efhalf_fit = data[f'fit_{fit_axis}']
npulsecalib_efhalf_fit_err = data[f'fit_err_{fit_axis}']

angle_err = npulsecalib_efhalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ef = np.reshape(npulsecalib_efhalf.cfg.device.qubit.pulses.pi_ef.half_gain, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ef.half_gain = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i] = int(round(new_amp_half_ef))
print(f'*Set pi/2 EF pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain = yaml_cfg.device.qubit.pulses.pi_ef.half_gain.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain}')
print(results_Qi)

fitparams [None, 0.017453292519943295]
Attempted to init fitparam 0 to 0.0, which is out of bounds 0.0 to 0.0. Instead init to 0.0
From amps: adjust amplitude to 6378 / 0.9989780638148239 = 6384.524576690065
From avgi: adjust amplitude to 6378 / 0.9989780638148239 = 6384.524576690065
From avgq: adjust amplitude to 6378 / 1.000193925472444 = 6376.76338314826
*Set pi/2 EF pulse gain for qubit 2 to 6385*
*New pi_ef.half_gain
 [[1261    0    0    0]
 [   0 1941    0    0]
 [   0    0 6385    0]
 [ 340  340    0  320]]
*New pi_ef.half_gain flattened
[1261, 0, 0, 0, 0, 1941, 0, 0, 0, 0, 6385, 0, 340, 340, 0, 320]
{'Res freq': 7702.328233333334, 'Res freq (yaml)': 452.32823333333357, 'Res kappa': 0.2819786993431654, 'Res chi shift (e - g)': -0.943708609271539, 'Res chi shift (f - g)': -1.9172185430463742, 'Res freq ef (yaml)': 450.4510147902872, 'Pi/2 EF pulse gain': 6385, 'Qubit EF freq': 4573.168610947125}


In [ ]:
if save_data: npulsecalib_efhalf.save_data()

## (Divide gain) Calibrate EF $\pi/2$ pulse

### Amplitude rabi for EF $\pi/2$ pulse (divide gain) (use this for RB)

In [ ]:
pi_lenEF = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.sigma, (4,4))[qubit_i, qubit_i]
# pi_len = 0.008
print('qubit', qubit_i, 'pi_lenEF', pi_lenEF)

In [ ]:
divide_len = False

In [ ]:
amprabi_4 = meas.AmplitudeRabiExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"amp_rabi_qubit{qubit_i}",
    config_file=config_path,
)
amprabi_4.cfg = AttrDict(deepcopy(yaml_cfg))

n_pulses = 1
span = min((32000, int(1.5*0.5*np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.gain, (4,4))[qubit_i, qubit_i])))
npts = 151
    
amprabi_4.cfg.expt = dict(
    start=0, # qubit gain [dac level]
    step=int(span/npts), # [dac level]
    expts=npts,
    reps=reps,
    # reps=5,
    rounds=rounds,
    sigma_test=pi_lenEF, # gaussian sigma for pulse length - overrides config [us]
    pulse_type='gauss',
    checkEF=True, 
    n_pulses=n_pulses,
    test_pi_half=True,
    divide_len=divide_len,
    qTest=qubit_i,
    qZZ=None,
)
amprabi_4.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
# # TESTING
# # amprabi_4 = meas.AmplitudeRabiExperiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_amp_rabi_qubit0.h5')
# amprabi_4.data = temp_data

amprabi_4.analyze(fit=True)

amprabi_4_fit, amprabi_4_fit_err = meas.fitting.get_best_fit(amprabi_4.data)

p = amprabi_4_fit
if p[2] > 180: p[2] = p[2] - 360
elif p[2] < -180: p[2] = p[2] + 360
if p[2] < 0: pi_gain = (1/2 - p[2]/180)/2/p[1]
else: pi_gain = (3/2 - p[2]/180)/2/p[1]

if pi_gain > 32000:
    amprabi_4.display(fit=False)
    print(f'WARNING: Fit pi/2 pulse gain is long ({pi_gain}), please double check!')
    pi_gain = 30000
else: amprabi_4.display(fit=True)

yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i] = int(pi_gain)
print(f'*Set EF pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 EF pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma}')
print(results_Qi)

In [ ]:
if save_data: saved_files.append(amprabi_4.save_data())

### EF $\pi/2$ error amplification (divide gain)

this is mostly only necessary for RB

In [ ]:
divide_len = False

In [ ]:
npulsecalib_efhalf = meas.NPulseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"NPulseExptEF_half_Q{qubit_i}",
    config_file=config_path,
)

npulsecalib_efhalf.cfg = AttrDict(deepcopy(yaml_cfg))
npulsecalib_efhalf.cfg.expt = dict(
    start=0, # number gate repetitions
    step=1,
    expts=10,
    reps=1000,
    loops=20,
    pulse_type='gauss',
    test_pi_half=True,
    divide_len=divide_len,
    error_amp=True,

    checkEF=True, 
    qTest=qubit_i,
    qZZ=None,

    readout_ge=False, # whether to readout at the g/e set point (True) or e/f set point (False)

    post_process='scale',
    singleshot_reps=15000,
)

print(npulsecalib_efhalf.cfg)
import Pyro4.util
try:
    npulsecalib_efhalf.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
# # TESTING
# temp_data, attrs = prev_data(expt_path, '00000_voltage_sweep_qubit_spectroscopy_qubit3.h5')
# data = lengthrabi.analyze(data=temp_data, fit=True)

data = npulsecalib_efhalf.data
npulsecalib_efhalf.analyze(fit=True, scale=None)
npulsecalib_efhalf.display(fit=True, scale=None)

# npulsecalib_efhalf_fit, npulsecalib_efhalf_fit_err = meas.fitting.get_best_fit(npulsecalib_efhalf.data)
fit_axis = 'avgi'
npulsecalib_efhalf_fit = data[f'fit_{fit_axis}']
npulsecalib_efhalf_fit_err = data[f'fit_err_{fit_axis}']

angle_err = npulsecalib_efhalf_fit[1]
amp_ratio = (90 - angle_err)/90
new_amp_half_ef = np.reshape(npulsecalib_efhalf.cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma, (4,4))[qubit_i, qubit_i] / amp_ratio

yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = np.reshape(yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma, (4,4))
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i] = int(round(new_amp_half_ef))
print(f'*Set pi/2 pulse gain for qubit {qubit_i} to {yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]}*')
results_Qi.update({'Pi/2 pulse gain':yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma[qubit_i, qubit_i]})
print(f'*New pi_ef.half_gain_pi_sigma\n', yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma)
yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma = yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma.flatten().astype(int).tolist()
print(f'*New pi_ef.half_gain_pi_sigma flattened\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma}')
print(results_Qi)

In [ ]:
if save_data: npulsecalib_efhalf.save_data()

## Resonator chi shift (e/f)

In [36]:
rspecEF_chi = meas.ResonatorSpectroscopyExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"resonator_spectroscopy_chi_qubit{qubit_i}",
    config_file=config_path,
)
rspecEF_chi.cfg = AttrDict(deepcopy(yaml_cfg))

span = 1.5 # MHz
npts = 151
rspecEF_chi.cfg.expt = dict(
    start=rspecEF_chi.cfg.device.readout.frequency[qubit_i]-abs(results_Qi['Res chi shift (e - g)'])-span, # [MHz]
    # start=rspecEF_chi.cfg.device.readout.frequency[qubit_i]-span/2, # [MHz]
    step=span/npts,
    expts=npts,
    reps=reps*rounds,
    pulse_e=True, # add ge pi pulse prior to measurement
    pulse_f=True, # add ef pi pulse prior to measurement
    qTest=qubit_i,
    qZZ=None,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# rspecEF_chi.cfg.device.readout.gain[qubit_i] /= 2
rspecEF_chi.cfg.device.readout.gain = rspecEF_chi.cfg.device.readout.gain_ef
# rspecEF_chi.cfg.device.readout.relax_delay = 100 # Wait time between experiments [us]
rspecEF_chi.go(analyze=False, display=False, progress=True, save=False)

  0%|          | 0/151 [00:00<?, ?it/s]

In [37]:
# # TESTING
# # rspecEF_chi = meas.ResonatorSpectroscopyExperiment(config_file=config_path,) 
# temp_data, attrs = prev_data(expt_path, '00001_resonator_spectroscopy_qubit2.h5')
# rspecEF_chi.data = temp_data

rspecEF_chi.data = rspecEF_chi.analyze(fit=True)
rspecEF_chi_fit = rspecEF_chi.data['fit']
rspecEF_chi.display(fit=True)
# res_freq_f = rspecEF_chi_fit[0] - yaml_cfg.hw.soc.dacs.readout.mixer_freq[qubit_i]
res_freq_f = rspecEF_chi.data['xpts'][np.argmin(rspecEF_chi.data['amps'])]
print('ef chi shift', yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_f - yaml_cfg.device.readout.frequency[qubit_i]))
results_Qi.update({'Res chi shift (f - g)': yaml_cfg.device.readout.lo_sideband[qubit_i]*(res_freq_f - yaml_cfg.device.readout.frequency[qubit_i])})
results_Qi.update({'Res freq ef (yaml)':res_freq_f})

print('Res freq ef:', res_freq_f)

# yaml_cfg.device.readout.frequency_ef = [0]*4
yaml_cfg.device.readout.frequency_ef[qubit_i] = res_freq_f
# res_freq_f = 6953.5
print(all_results)

electrical delay phase correction (no mux support) (deg) -6.43294603013481
param guess [6969.6, 34848.0, 34848.0, 0, -0.038507939283982044, 0.9015028307304147]
param bounds ([6968.864900662252, 0, 0, -6.283185307179586, -0.05776190892597306, -inf], [6970.335099337749, 1000000000.0, 100000000.0, 6.283185307179586, 0.05776190892597306, 1.0233088387647726])
popt [ 6.96958147e+03  5.10160795e+04  8.16981033e+04  1.56183812e-01
 -2.48879274e-02 -2.49383477e-02]

Freq with minimum transmission: -280.39006622516524
Freq with maximum transmission: -281.135099337748
From fit:
	f0: 6969.581466448218
	Qi: 51016.07954771201 	 kappa_i/2pi: 0.13661538730999553
	Qe: 81698.10330206447 	 kappa_e/2pi: 0.08530897517509566
	Q0: 31405.211164756332
	kappa [MHz]: 0.2219243624850912
	phi [radians]: 0.15618381215071592
baseline 1547.2960155922258
ef chi shift -1.4801324503311548
Res freq ef: -280.3801324503308
[{'Res chi shift (e - g)': -1.0033112582781314, 'Res chi shift (f - g)': -1.939084045035088, 'Res fre

In [ ]:
if save_data: saved_files.append(rspecEF_chi.save_data())

## $T_1$ EF

In [137]:
t1EF = meas.T1Experiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"t1EF_qubit{qubit_i}",
    config_file=config_path,
)
t1EF.cfg = AttrDict(deepcopy(yaml_cfg))

span = 100
npts = 121

t1EF.cfg.expt = dict(
    start=0, # wait time [us]
    step=span/npts,
    expts=npts,
    reps=reps*rounds*4,
    rounds=1,
    qTest=qubit_i,
    qZZ=None,
    # qZZ=0,
    checkEF=True,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# for q in range(4):
#     if q != qubit_i:
#         t1EF.cfg.device.readout.gain[q] = 1e-4

t1EF.cfg.device.readout.relax_delay = span # Wait time between experiments [us]
t1EF.go(analyze=False, display=False, progress=True, save=False)

  0%|          | 0/484000 [00:00<?, ?it/s]

In [138]:
# # TESTING
# # t1EF = meas.T1Experiment(config_file=config_path,)
# temp_data, attrs = prev_data(expt_path, '00000_t1EF_qubit1.h5')
# t1EF.data = temp_data

t1EF.analyze(fit_log=False)

t1EF_fit, t1EF_fit_err = meas.fitting.get_best_fit(t1EF.data) #, fitter.expfunc)

t1EF.display(fit=True, fit_log=False)
t1EF_qubit = t1EF_fit[3]
yaml_cfg.device.readout.relax_delay[qubit_i] = np.round(4*t1EF_qubit)
print(f'*Set readout {qubit_i} relax delay to {yaml_cfg.device.readout.relax_delay[qubit_i]}*')
results_Qi.update({'T1_EF':t1EF_qubit, 'T1_EF err':np.sqrt(t1EF_fit_err[3][3])})
print(f'qubit {qubit_i}', results_Qi)

best 0 113353.36745871502
Fit T1 amps [us]: 21.222524604661047
Fit T1 avgi [us]: 20.052374595554404
Fit T1 avgq [us]: 29.517732509264814
*Set readout 3 relax delay to 85.0*
qubit 3 {'Pi pulse gain': 14360, 'Qubit freq': 4381.135527846996, 'Res freq': 6971.153333333334, 'Res freq (yaml)': -228.84666666666666, 'Res kappa': 0.20149515995222483, 'Pi/2 pulse gain': 3120, 'Single shot g-e fidelity (%)': 2.2249999999999934, 'T1_EF': 21.222524604661047, 'T1_EF err': 1.2097921499036397}


In [ ]:
if save_data: saved_files.append(t1.save_data())

# Single Shot

In [18]:
import yaml
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

## Single shot optimization

In [ ]:
results_Qi['Res kappa'] = 0.2
# print(results_Qi['Res kappa'])

In [76]:
qubit_i = 3

In [ ]:
shotopt = meas.SingleShotOptExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_opt_qubit{qubit_i}",
    config_file=config_path,
)
shotopt.cfg = AttrDict(deepcopy(yaml_cfg))

# span_f = 2*abs(results_Qi['Res chi shift (f - g)']) # MHz
# span_f = 2*abs(results_Qi['Res chi shift (e - g)']) # MHz

span_f = 1.0*abs(results_Qi['Res kappa']) # MHz
npts_f = 15
# span_f = 0
# npts_f = 1

span_gain = 0.4
npts_gain = 8
# span_gain = 0.0
# npts_gain = 8

# span_len = 5 # us
# npts_len = 10
span_len = 0 # us
npts_len = 1

shotopt.cfg.expt = dict(
    reps=15000,
    qTest=qubit_i,

    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] - span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] + 0.25*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i],
    start_f=shotopt.cfg.device.readout.frequency[qubit_i] - span_f/2, # [MHz]
    step_f=span_f/npts_f,
    expts_f=npts_f,

    # start_gain=shotopt.cfg.device.readout.gain[qubit_i]-span_gain/2,
    start_gain=0.2,
    step_gain=span_gain/npts_gain,
    expts_gain=npts_gain + 1,
    # start_gain=0.2,
    # step_gain=0,
    # expts_gain=1,

    start_len=shotopt.cfg.device.readout.readout_length[qubit_i],
    step_len=span_len/npts_len,
    expts_len=npts_len,
    # start_len=7,
    # step_len=span_len/npts_len,
    # expts_len=npts_len + 1,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

print(shotopt.cfg)

shotopt.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
imax = shotopt.analyze()
shotopt.display()
print(shotopt.data['fid'])

freq_opt = shotopt.data['fpts'][imax[0]]
gain_opt = shotopt.data['gainpts'][imax[1]]
len_opt = shotopt.data['lenpts'][imax[2]]

In [ ]:

# Using readlines()
file1 = open('C:\\Users\\slab\\Downloads\\singleshot_opt.txt', 'r')
lines = file1.readlines()
freqs = []
gains = []
lengths = []
fids = []

line = lines[0]
words = line.split()
freq = float(words[1][:-1])
freqs.append(freq)
gain = float(words[3][:-1])
gains.append(gain)
length = float(words[5])
lengths.append(length)

fid_line = lines[1]
fids.append(float(fid_line.split()[3]))
 
# Strips the newline character
for i in range(2, len(lines), 2):
    line = lines[i]
    words = line.split()
    freq = float(words[1][:-1])
    if freq not in freqs: freqs.append(freq)
    gain = float(words[3][:-1])
    if gain not in gains: gains.append(gain)
    length = float(words[5])
    if length not in lengths: lengths.append(length)

    fid_line = lines[i+1]
    fids.append(float(fid_line.split()[3]))
    continue

fids = np.append(fids, [0]*22)
fids = np.reshape(fids, newshape=(len(freqs), len(gains), len(lengths)))

# lenpts = [data['lenpts'][0]]
for g_ind, gain in enumerate(gains):
    for l_ind, l in enumerate(lengths):
        plt.plot(freqs, fids[:,g_ind, l_ind], 'o-', label=f'gain: {gain:.2}, len [us]: {l}')
plt.xlabel('Frequency [MHz]')
plt.ylabel(f'Fidelity [%]')
plt.legend()
plt.show()


In [ ]:
if save_data: saved_files.append(shotopt.save_data())

next qubit

In [ ]:
qubit_i = 2
results_Qi = all_results[qubit_i]
results_Qi['Res kappa'] = 0.2
# print(results_Qi['Res kappa'])

In [ ]:
shotopt = meas.SingleShotOptExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_opt_qubit{qubit_i}",
    config_file=config_path,
)
shotopt.cfg = AttrDict(deepcopy(yaml_cfg))

# span_f = 2*abs(results_Qi['Res chi shift (f - g)']) # MHz
# span_f = 2*abs(results_Qi['Res chi shift (e - g)']) # MHz

# span_f = 2*abs(results_Qi['Res kappa']) # MHz
# npts_f = 15

span_f = 3*abs(results_Qi['Res kappa']) # MHz
npts_f = 20

# span_gain = 0.85
# npts_gain = 5
span_gain = 0.6
npts_gain = 6

span_len = 0 # us
npts_len = 1

shotopt.cfg.expt = dict(
    reps=20000,
    qTest=qubit_i,

    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] - 0.75*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] + 0.25*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i],
    start_f=shotopt.cfg.device.readout.frequency[qubit_i] - span_f/2, # [MHz]
    step_f=span_f/npts_f,
    expts_f=npts_f,

    # start_gain=int(shotopt.cfg.device.readout.gain[qubit_i]-span_gain/2),
    start_gain=0.1,
    # start_gain=0.10,
    step_gain=span_gain/npts_gain,
    expts_gain=npts_gain + 1,

    start_len=5,
    # step_len=span_len/npts_len,
    # expts_len=npts_len + 1,
    step_len=0,
    expts_len=1,
)

print(shotopt.cfg)

shotopt.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
imax = shotopt.analyze()
shotopt.display()
print(shotopt.data['fid'])

freq_opt = shotopt.data['fpts'][imax[0]]
gain_opt = shotopt.data['gainpts'][imax[1]]
len_opt = shotopt.data['lenpts'][imax[2]]

In [ ]:
if save_data: saved_files.append(shotopt.save_data())

next qubit

In [ ]:
qubit_i = 3
results_Qi = all_results[qubit_i]
results_Qi['Res kappa'] = 0.2
# print(results_Qi['Res kappa'])

In [ ]:
shotopt = meas.SingleShotOptExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_opt_qubit{qubit_i}",
    config_file=config_path,
)
shotopt.cfg = AttrDict(deepcopy(yaml_cfg))

# span_f = 2*abs(results_Qi['Res chi shift (f - g)']) # MHz
# span_f = 2*abs(results_Qi['Res chi shift (e - g)']) # MHz

# span_f = 2*abs(results_Qi['Res kappa']) # MHz
# npts_f = 15

span_f = 3*abs(results_Qi['Res kappa']) # MHz
npts_f = 20

# span_gain = 0.85
# npts_gain = 5
span_gain = 0.1
npts_gain = 8

span_len = 0 # us
npts_len = 1

shotopt.cfg.expt = dict(
    reps=15000,
    qTest=qubit_i,

    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] - 0.75*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i] + 0.25*span_f,
    # start_f=shotopt.cfg.device.readout.frequency[qubit_i],
    start_f=shotopt.cfg.device.readout.frequency[qubit_i] - span_f/2, # [MHz]
    step_f=span_f/npts_f,
    expts_f=npts_f,

    # start_gain=int(shotopt.cfg.device.readout.gain[qubit_i]-span_gain/2),
    start_gain=0.01,
    # start_gain=0.10,
    step_gain=span_gain/npts_gain,
    expts_gain=npts_gain + 1,

    start_len=8.0,
    # step_len=span_len/npts_len,
    # expts_len=npts_len + 1,
    step_len=0,
    expts_len=1,
)

print(shotopt.cfg)

shotopt.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
imax = shotopt.analyze()
shotopt.display()
print(shotopt.data['fid'])

freq_opt = shotopt.data['fpts'][imax[0]]
gain_opt = shotopt.data['gainpts'][imax[1]]
len_opt = shotopt.data['lenpts'][imax[2]]

In [ ]:
if save_data: saved_files.append(shotopt.save_data())

## Best readout parameters

In [19]:
qubit_i = 3

In [134]:
shot = meas.HistogramExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"single_shot_qubit{qubit_i}",
    config_file=config_path,
)
shot.cfg = AttrDict(deepcopy(yaml_cfg))

shot.cfg.expt = dict(
    reps=8000,
    check_f=True,
    qTest=qubit_i,

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)

# shot.cfg.device.readout.frequency=freq_opt
# shot.cfg.device.readout.gain[qubit_i]=gain_opt
# print(freq_opt, gain_opt)
# shot.cfg.device.readout.readout_length = len_opt # [us]

# shot.cfg.device.readout.frequency=-807.4996158081035
# shot.cfg.device.readout.gain[qubit_i] = 0.85
# shot.cfg.device.readout.readout_length = 2.0 # [us]
# shot.cfg.device.readout.relax_delay = 70 # Wait time between experiments [us]
print(shot.cfg)

shot.go(analyze=False, display=False, progress=True, save=False)

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.975, 5795.761666663333, 502.368233333334, -228.9], 'gain': [0.5, 0.6, 0.4, 0.3], 'readout_length': [5, 20, 10, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.144451095202, 4105.783958808882, 4111.0028964412795, 4110.5966963096635, 3442.9795552267146, 3448.3315918098933, 3447.233490336077, 3447.0285717924894, 4762.767009306428, 4761.793655204115, 4762.902435440665, 4762.367822139689, 4380.662337410765, 4379.890920412824, 4380.6668186549305, 4381.242475074487], 'f_ef': [3885.3117884920853, 0.0, 0.0, 0.0, 0.0, 3347.8480888529375, 0.0, 0.0, 4571.9056482340775, 4571.070566218902, 4572.9717022650675, 0.0, 4211.5805515224565, 4207.169953136575, 0.0, 4209.889752285018], 'f_EgGf': [2686.53594683, 0, 2017.3, 2414.92850999], 'f_EgGf_Q_half':

  0%|          | 0/8000 [00:00<?, ?it/s]

  0%|          | 0/8000 [00:00<?, ?it/s]

  0%|          | 0/8000 [00:00<?, ?it/s]

In [135]:
# # TESTING
# shot = meas.HistogramExperiment(config_file=config_path)
# temp_data, attrs = prev_data(expt_path, '00001_single_shot_qubit0.h5')
# data = shot.analyze(data=temp_data)
# shot.display(data=data)

shot.analyze(verbose=False)
shot.display(verbose=True, amplitude_mode=shot.cfg.expt.full_mux_expt)
results_Qi.update({'Single shot g-e fidelity (%)': 100*shot.data['fids'][0]})

g unrotated averages:
I -23.483399257526443 +/- 21.860967471456476 	 Q -55.07453519121238 +/- 20.373169359948697 	 Amp 63.70580234972398 +/- 20.47469518092067
Rotated (theta=-0.39449266323091886):
I -42.8470182984508 +/- 21.74272316073474 	 Q -41.81874567880661 +/- 20.499315064481536 	 Amp 59.872151015446946 +/- 20.47469518092067
e unrotated averages:
I -23.019201535801464 +/- 21.882729577960223 	 Q -55.40713430634662 +/- 20.60830272348304 	 Amp 63.81193805008286 +/- 20.770814346207487
Rotated (theta=-0.39449266323091886):
I -42.546305927579496 +/- 21.859458416844202 	 Q -42.30420810397226 +/- 20.632985063899813 	 Amp 59.998618078980606 +/- 20.770814346207487
f unrotated averages:
I -22.835162683075673 +/- 21.586668324180526 	 Q -54.80466364930838 +/- 20.273200951245645 	 Amp 63.14813294218095 +/- 20.35426354700071
Rotated (theta=-0.39449266323091886):
I -42.144849297721414 +/- 21.550150440184776 	 Q -41.81874567880662 +/- 20.312014724127447 	 Amp 59.371675169868304 +/- 20.354263547000

In [ ]:
if save_data: saved_files.append(shot.save_data())

In [ ]:
# TESTING
shot = meas.HistogramExperiment(config_file=config_path)
temp_data, attrs = prev_data('S:\\Connie\\experiments\\qramLL_4QR2\\data\\data_240617\\00000_single_shot_qubit2.h5')
shot.cfg = AttrDict(attrs['config'])
shot.data = temp_data

In [ ]:
# Checking for drift vs. shot number

data = shot.data

Ies = data['Ie']
Qes = data['Qe']
Igs = data['Ig']
Qgs = data['Qg']

plot_e = False

if plot_e:
    i, q = Ies, Qes
else:
    i, q = Igs, Qgs

amps = np.abs(i+1j*q)
phases = np.angle(i+1j*q)

# plt.plot(amps, '.', label='amps')
# plt.ylabel('Amps [ADC units]')
# plt.xlabel('Shot number')
# plt.legend()
# plt.show()

plt.title(f'Q{shot.cfg.expt.qTest} {"e" if plot_e else "g"}')
plt.plot(phases, '.', label='phases')
plt.axhline(0, color='r')
# plt.axhline(1.5, color='r')
plt.ylabel('Phases [radians]')
plt.xlabel('Shot number')
plt.legend()
plt.show()

plt.plot(i, '.', label='i')
plt.ylabel('I [ADC units]')
plt.xlabel('Shot number')
plt.legend()
plt.show()

plt.plot(q, '.', label='Q')
plt.ylabel('Q [ADC units]')
plt.xlabel('Shot number')
plt.legend()
plt.show()

# Results

In [74]:
print(yaml_cfg)
for Qi, results in enumerate(all_results):
    print(f'Qubit {Qi}')
    for key, value in results.items():
        print(f'\t{key}: {value}')
    print()

for saved_file in saved_files:
    print(saved_file)

print(f'f_ge\n{yaml_cfg.device.qubit.f_ge}')
print(f'pi_ge.gain\n{yaml_cfg.device.qubit.pulses.pi_ge.gain}')
print(f'pi_ge.half_gain\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain}')
print(f'pi_ge.half_gain_pi_sigma\n{yaml_cfg.device.qubit.pulses.pi_ge.half_gain_pi_sigma}')
print(f'f_ef\n{yaml_cfg.device.qubit.f_ef}')
print(f'pi_ef.gain\n{yaml_cfg.device.qubit.pulses.pi_ef.gain}')
print(f'pi_ef.half_gain\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain}')
print(f'pi_ef.half_gain_pi_sigma\n{yaml_cfg.device.qubit.pulses.pi_ef.half_gain_pi_sigma}')
print(f'readout.frequency_ef\n{yaml_cfg.device.readout.frequency_ef}')

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.9452717544333, 5795.711666663333, 502.368233333334, -228.85410723], 'gain': [0.3, 0.3, 0.85, 0.85], 'readout_length': [15.0, 10.0, 5.0, 15.0], 'frequency_ef': [-391.80446161500305, 5794.287237348879, 500.35983051204073, -230.41370987900663], 'readout_length_ef': [10.0, 7.0, 4.0, 8.0], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.097866562327, 4105.867592682015, 4110.994688934281, 4110.579369837695, 3443.482349236793, 3448.1402562265434, 3447.706823169816, 3447.5052085761054, 4762.743580751455, 4761.745710010409, 4762.909473544484, 4762.370786469688, 4380.292461267258, 4379.535103914169, 4380.328267473819, 4381.248121020163], 'f_ef': [3884.6269161242526, 0.0, 0.0, 0.0, 0.0, 3348.085433174199, 0.0, 0.0, 4571.9056482340775, 4570.996526541673, 4573.058329968845, 0.0, 4209.8719428597715, 4205.61697813965, 0.0, 4208.990881064611], 'f_EgGf': [2686.53594683, 0, 2017.3, 2414.92850999], '